In [1]:
# Cell 1: Install Dependencies (Final)
print("Installing all required packages...")

# Set up environment variables from Colab Secrets
import os
from google.colab import userdata

os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['JWT_SECRET_KEY'] = userdata.get('JWT_SECRET_KEY')
os.environ['NGROK_TOKEN'] = userdata.get('NGROK_TOKEN')
os.environ['EMAIL_ADDRESS'] = userdata.get('EMAIL_ADDRESS')
os.environ['EMAIL_PASSWORD'] = userdata.get('EMAIL_PASSWORD')

# Install all required packages
!pip install --upgrade pip

# PyTorch
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Transformers, Accelerate, and BitsAndBytes for 4-bit loading
!pip install transformers accelerate bitsandbytes
!pip install reportlab
# Streamlit and utilities (radon removed)
!pip install streamlit pyngrok PyJWT bcrypt pandas graphviz plotly-express psutil streamlit-autorefresh numpy

print("All dependencies installed successfully.")

Installing all required packages...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 73.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 64.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 101.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 86.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 700.8/700.8 kB 39.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [streamlit-autorefresh]
All dependencies installed successfully.


In [5]:
%%writefile app.py

import streamlit as st
import sqlite3
import bcrypt
import jwt
import os
import graphviz
import ast
import torch
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import time
import gc
import psutil
import json
import numpy as np
import smtplib
import random
import requests
import uuid
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from datetime import datetime, timedelta
from streamlit_autorefresh import st_autorefresh
import base64
from io import BytesIO
import tempfile
import html
import threading

# Try to import reportlab, but provide fallback if not available
try:
    from reportlab.lib.pagesizes import letter, A4
    from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, PageBreak
    from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
    from reportlab.lib.units import inch
    from reportlab.lib import colors
    from reportlab.pdfgen import canvas
    from reportlab.lib.utils import simpleSplit
    REPORTLAB_AVAILABLE = True
except ImportError:
    REPORTLAB_AVAILABLE = False
    st.warning("ReportLab not available. PDF export will use fallback method.")

# --- 1. CONFIGURATION & SECRETS ---

HF_TOKEN = os.environ.get('HF_TOKEN')
JWT_SECRET = os.environ.get('JWT_SECRET_KEY', "dummy_jwt_secret_key_2024")
JWT_REFRESH_SECRET = os.environ.get('JWT_REFRESH_SECRET', "dummy_refresh_secret_2024")
NGROK_TOKEN = os.environ.get('NGROK_TOKEN', "dummy_ngrok")

# Email configuration
SMTP_SERVER = os.environ.get('SMTP_SERVER', 'smtp.gmail.com')
SMTP_PORT = int(os.environ.get('SMTP_PORT', 587))
EMAIL_ADDRESS = os.environ.get('EMAIL_ADDRESS', '')
EMAIL_PASSWORD = os.environ.get('EMAIL_PASSWORD', '')

DB_NAME = "users.db"

# --- GLOBAL MODEL CACHE ---
# This will be shared across all users in the same process
MODEL_CACHE = {
    'code_models': {},
    'chat_models': {},
    'loading_status': {},  # Track which models are being loaded
    'loaded_at': None
}

# --- 2. DATABASE UTILITIES ---

def init_db():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()

    # Users table with email
    c.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE NOT NULL,
        email TEXT UNIQUE NOT NULL,
        password_hash TEXT NOT NULL,
        security_question TEXT NOT NULL,
        security_answer_hash TEXT NOT NULL,
        role TEXT NOT NULL DEFAULT 'user',
        created_at TEXT NOT NULL,
        is_active INTEGER DEFAULT 1
    )
    ''')

    # OTP table for password recovery
    c.execute('''
    CREATE TABLE IF NOT EXISTS otp_codes (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        email TEXT NOT NULL,
        otp_code TEXT NOT NULL,
        created_at TEXT NOT NULL,
        expires_at TEXT NOT NULL,
        is_used INTEGER DEFAULT 0
    )
    ''')

    # Conversations table
    c.execute('''
    CREATE TABLE IF NOT EXISTS conversations (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER NOT NULL,
        title TEXT NOT NULL,
        created_at TEXT NOT NULL,
        updated_at TEXT NOT NULL,
        is_shared INTEGER DEFAULT 0,
        share_id TEXT UNIQUE,
        FOREIGN KEY (user_id) REFERENCES users (id)
    )
    ''')

    # Messages table
    c.execute('''
    CREATE TABLE IF NOT EXISTS messages (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        conversation_id INTEGER NOT NULL,
        role TEXT NOT NULL,
        content TEXT NOT NULL,
        model_name TEXT,
        language TEXT,
        timestamp TEXT NOT NULL,
        response_time REAL,
        feedback_score INTEGER DEFAULT 0,
        feedback_comment TEXT,
        tokens_used INTEGER DEFAULT 0,
        FOREIGN KEY (conversation_id) REFERENCES conversations (id)
    )
    ''')

    # Activity log for analytics
    c.execute('''
    CREATE TABLE IF NOT EXISTS activity_log (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        activity_type TEXT NOT NULL,
        timestamp TEXT NOT NULL,
        details TEXT,
        module TEXT,
        FOREIGN KEY (user_id) REFERENCES users (id)
    )
    ''')

    # History entries table for Code Generator and Explainer
    c.execute('''
    CREATE TABLE IF NOT EXISTS history_entries (
        id TEXT PRIMARY KEY,
        user_id INTEGER NOT NULL,
        type TEXT NOT NULL CHECK (type IN ('generator', 'explainer')),
        created_at TEXT NOT NULL,
        inputs_json TEXT NOT NULL,
        outputs_text TEXT,
        outputs_code TEXT,
        metadata_json TEXT,
        FOREIGN KEY (user_id) REFERENCES users (id)
    )
    ''')

    # Create indexes for performance
    c.execute('CREATE INDEX IF NOT EXISTS idx_history_user_type ON history_entries(user_id, type)')
    c.execute('CREATE INDEX IF NOT EXISTS idx_history_created_at ON history_entries(created_at)')
    c.execute('CREATE INDEX IF NOT EXISTS idx_history_user_created ON history_entries(user_id, created_at)')

    c.execute("SELECT COUNT(*) FROM users WHERE role='admin'")
    admin_count = c.fetchone()[0]

    # Create default admin if no users exist
    c.execute("SELECT COUNT(*) FROM users")
    user_count = c.fetchone()[0]

    conn.commit()
    conn.close()

    return user_count == 0, admin_count

def hash_text(text):
    salt = bcrypt.gensalt()
    return bcrypt.hashpw(text.encode('utf-8'), salt)

def check_hash(text, hashed):
    return bcrypt.checkpw(text.encode('utf-8'), hashed)

def add_user(username, email, password, question, answer, role='user'):
    try:
        conn = sqlite3.connect(DB_NAME)
        c = conn.cursor()

        # Check admin limit
        if role == 'admin':
            c.execute("SELECT COUNT(*) FROM users WHERE role='admin'")
            admin_count = c.fetchone()[0]
            if admin_count >= 2:
                conn.close()
                return False, "Maximum admin limit (2) reached"

        password_hash = hash_text(password)
        answer_hash = hash_text(answer)
        created_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        c.execute(
            "INSERT INTO users (username, email, password_hash, security_question, security_answer_hash, role, created_at) VALUES (?, ?, ?, ?, ?, ?, ?)",
            (username, email, password_hash, question, answer_hash, role, created_at)
        )
        conn.commit()
        return True, "User created successfully"
    except sqlite3.IntegrityError as e:
        if "username" in str(e):
            return False, "Username already exists"
        elif "email" in str(e):
            return False, "Email already exists"
        return False, "Database integrity error"
    finally:
        conn.close()

def verify_user(username, password):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT password_hash, role, email, id FROM users WHERE username=? AND is_active=1", (username,))
    user = c.fetchone()
    conn.close()
    if user:
        password_hash, role, email, user_id = user
        if check_hash(password, password_hash):
            return {"username": username, "role": role, "email": email, "user_id": user_id}
    return None

def update_user_password(username, new_password):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    new_password_hash = hash_text(new_password)
    c.execute("UPDATE users SET password_hash = ? WHERE username = ?", (new_password_hash, username))
    conn.commit()
    conn.close()

def get_user_by_email(email):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT username, security_question FROM users WHERE email=?", (email,))
    user = c.fetchone()
    conn.close()
    return user

def store_otp(email, otp_code):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    created_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    expires_at = (datetime.now() + timedelta(minutes=10)).strftime("%Y-%m-%d %H:%M:%S")

    # Invalidate previous OTPs
    c.execute("UPDATE otp_codes SET is_used=1 WHERE email=?", (email,))

    c.execute(
        "INSERT INTO otp_codes (email, otp_code, created_at, expires_at) VALUES (?, ?, ?, ?)",
        (email, otp_code, created_at, expires_at)
    )
    conn.commit()
    conn.close()

def verify_otp(email, otp_code):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute(
        "SELECT id FROM otp_codes WHERE email=? AND otp_code=? AND is_used=0 AND expires_at > ?",
        (email, otp_code, datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    )
    result = c.fetchone()
    if result:
        c.execute("UPDATE otp_codes SET is_used=1 WHERE id=?", (result[0],))
        conn.commit()
        conn.close()
        return True
    conn.close()
    return False

def get_all_users():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT id, username, email, role, created_at, is_active FROM users ORDER BY created_at DESC")
    users = c.fetchall()
    conn.close()
    return users

def delete_user(user_id):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("UPDATE users SET is_active=0 WHERE id=?", (user_id,))
    conn.commit()
    conn.close()

def promote_to_admin(user_id):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()

    # Check admin limit
    c.execute("SELECT COUNT(*) FROM users WHERE role='admin' AND is_active=1")
    admin_count = c.fetchone()[0]

    if admin_count >= 2:
        conn.close()
        return False, "Maximum admin limit (2) reached"

    c.execute("UPDATE users SET role='admin' WHERE id=?", (user_id,))
    conn.commit()
    conn.close()
    return True, "User promoted to admin"

def get_user_details(username):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT username, email, security_question, security_answer_hash FROM users WHERE username=?", (username,))
    user = c.fetchone()
    conn.close()
    if user:
        return {"username": user[0], "email": user[1], "question": user[2], "answer_hash": user[3]}
    return None

def get_user_id(username):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT id FROM users WHERE username = ?", (username,))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

# --- HISTORY MANAGEMENT FUNCTIONS ---

def create_history_entry(user_id, entry_type, inputs, outputs_text=None, outputs_code=None, metadata=None):
    """
    Create a history entry for Code Generator or Explainer
    """
    try:
        conn = sqlite3.connect(DB_NAME)
        c = conn.cursor()

        entry_id = str(uuid.uuid4())
        created_at = datetime.now().isoformat()

        inputs_json = json.dumps(inputs)
        metadata_json = json.dumps(metadata) if metadata else "{}"

        c.execute('''
            INSERT INTO history_entries
            (id, user_id, type, created_at, inputs_json, outputs_text, outputs_code, metadata_json)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', (entry_id, user_id, entry_type, created_at, inputs_json, outputs_text, outputs_code, metadata_json))

        conn.commit()
        conn.close()

        # Log activity
        log_activity(user_id, f"create_{entry_type}_history", "history", f"Created {entry_type} entry: {entry_id}")
        add_system_log(f"User {user_id} created {entry_type} history entry: {entry_id}", "INFO")

        return entry_id
    except Exception as e:
        print(f"Error creating history entry: {e}")
        return None

def get_user_history_entries(user_id, entry_type, page=1, page_size=20, search_query=None, time_filter=None):
    """
    Get paginated history entries for a user with optional search and time filtering
    """
    try:
        conn = sqlite3.connect(DB_NAME)
        c = conn.cursor()

        # Build query with filters
        query = '''
            SELECT id, type, created_at, inputs_json, outputs_text, outputs_code, metadata_json
            FROM history_entries
            WHERE user_id = ? AND type = ?
        '''
        params = [user_id, entry_type]

        # Add time filter
        if time_filter:
            if time_filter == '24h':
                cutoff_date = (datetime.now() - timedelta(hours=24)).isoformat()
            elif time_filter == '7d':
                cutoff_date = (datetime.now() - timedelta(days=7)).isoformat()
            elif time_filter == '30d':
                cutoff_date = (datetime.now() - timedelta(days=30)).isoformat()
            else:
                cutoff_date = None

            if cutoff_date:
                query += " AND created_at >= ?"
                params.append(cutoff_date)

        # Add search filter
        if search_query:
            query += " AND (inputs_json LIKE ? OR outputs_text LIKE ? OR outputs_code LIKE ?)"
            search_pattern = f"%{search_query}%"
            params.extend([search_pattern, search_pattern, search_pattern])

        # Add ordering and pagination
        query += " ORDER BY created_at DESC LIMIT ? OFFSET ?"
        params.extend([page_size, (page - 1) * page_size])

        c.execute(query, params)
        entries = c.fetchall()

        # Get total count for pagination
        count_query = '''
            SELECT COUNT(*) FROM history_entries
            WHERE user_id = ? AND type = ?
        '''
        count_params = [user_id, entry_type]

        if time_filter and cutoff_date:
            count_query += " AND created_at >= ?"
            count_params.append(cutoff_date)

        if search_query:
            count_query += " AND (inputs_json LIKE ? OR outputs_text LIKE ? OR outputs_code LIKE ?)"
            count_params.extend([search_pattern, search_pattern, search_pattern])

        c.execute(count_query, count_params)
        total_count = c.fetchone()[0]

        conn.close()

        # Parse JSON fields
        parsed_entries = []
        for entry in entries:
            entry_id, entry_type, created_at, inputs_json, outputs_text, outputs_code, metadata_json = entry
            parsed_entries.append({
                'id': entry_id,
                'type': entry_type,
                'created_at': created_at,
                'inputs': json.loads(inputs_json),
                'outputs_text': outputs_text,
                'outputs_code': outputs_code,
                'metadata': json.loads(metadata_json) if metadata_json else {}
            })

        return {
            'entries': parsed_entries,
            'total_count': total_count,
            'page': page,
            'page_size': page_size,
            'total_pages': (total_count + page_size - 1) // page_size
        }
    except Exception as e:
        print(f"Error getting history entries: {e}")
        return {'entries': [], 'total_count': 0, 'page': page, 'page_size': page_size, 'total_pages': 0}

def get_history_entry(entry_id, user_id):
    """
    Get a specific history entry with user access control
    """
    try:
        conn = sqlite3.connect(DB_NAME)
        c = conn.cursor()

        c.execute('''
            SELECT id, type, created_at, inputs_json, outputs_text, outputs_code, metadata_json
            FROM history_entries
            WHERE id = ? AND user_id = ?
        ''', (entry_id, user_id))

        entry = c.fetchone()
        conn.close()

        if entry:
            entry_id, entry_type, created_at, inputs_json, outputs_text, outputs_code, metadata_json = entry
            return {
                'id': entry_id,
                'type': entry_type,
                'created_at': created_at,
                'inputs': json.loads(inputs_json),
                'outputs_text': outputs_text,
                'outputs_code': outputs_code,
                'metadata': json.loads(metadata_json) if metadata_json else {}
            }
        return None
    except Exception as e:
        print(f"Error getting history entry: {e}")
        return None

def delete_history_entry(entry_id, user_id):
    """
    Delete a history entry with user access control
    """
    try:
        conn = sqlite3.connect(DB_NAME)
        c = conn.cursor()

        c.execute('DELETE FROM history_entries WHERE id = ? AND user_id = ?', (entry_id, user_id))
        conn.commit()
        conn.close()

        if c.rowcount > 0:
            log_activity(user_id, "delete_history", "history", f"Deleted history entry: {entry_id}")
            add_system_log(f"User {user_id} deleted history entry: {entry_id}", "INFO")
            return True
        return False
    except Exception as e:
        print(f"Error deleting history entry: {e}")
        return False

# --- PDF GENERATION FUNCTIONS ---

def create_pdf_download(entries, entry_type, user_id, username):
    """
    Create a PDF file for history entries download
    """
    try:
        # Create a temporary file
        with tempfile.NamedTemporaryFile(delete=False, suffix='.pdf') as tmp_file:
            filename = tmp_file.name

        # Create PDF document
        doc = SimpleDocTemplate(filename, pagesize=A4, topMargin=1*inch, bottomMargin=1*inch)
        styles = getSampleStyleSheet()

        # Custom styles
        title_style = ParagraphStyle(
            'CustomTitle',
            parent=styles['Heading1'],
            fontSize=16,
            spaceAfter=30,
            alignment=1  # Center
        )

        heading_style = ParagraphStyle(
            'CustomHeading',
            parent=styles['Heading2'],
            fontSize=12,
            spaceAfter=12,
            textColor=colors.darkblue
        )

        metadata_style = ParagraphStyle(
            'Metadata',
            parent=styles['Normal'],
            fontSize=9,
            textColor=colors.gray
        )

        code_style = ParagraphStyle(
            'Code',
            parent=styles['Code'],
            fontSize=8,
            fontName='Courier',
            leftIndent=10,
            rightIndent=10,
            spaceAfter=12
        )

        # Build story (content)
        story = []

        # Header
        story.append(Paragraph("CodeGenie History Export", title_style))
        story.append(Paragraph(f"Type: {entry_type.title()} History", styles['Heading2']))
        story.append(Paragraph(f"User: {username} (ID: {user_id})", styles['Normal']))
        story.append(Paragraph(f"Export Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S UTC')}", styles['Normal']))
        story.append(Spacer(1, 20))

        # Add each entry
        for i, entry in enumerate(entries):
            if i > 0:
                story.append(PageBreak())

            # Entry header
            story.append(Paragraph(f"Entry {i+1}: {entry['type'].title()} - {entry['id'][:8]}", heading_style))
            story.append(Paragraph(f"Created: {entry['created_at']}", metadata_style))
            story.append(Spacer(1, 10))

            # Inputs section
            story.append(Paragraph("Inputs:", styles['Heading3']))
            inputs_text = json.dumps(entry['inputs'], indent=2)
            story.append(Paragraph(f"<pre>{html.escape(inputs_text)}</pre>", code_style))
            story.append(Spacer(1, 10))

            # Outputs section
            story.append(Paragraph("Outputs:", styles['Heading3']))
            if entry['outputs_code']:
                # Code output
                code_lines = entry['outputs_code'].split('\n')
                # Truncate very long code for PDF
                if len(code_lines) > 100:
                    code_lines = code_lines[:100] + ['\n// ... (truncated for PDF) ...']
                truncated_code = '\n'.join(code_lines)
                story.append(Paragraph(f"<pre>{html.escape(truncated_code)}</pre>", code_style))
            elif entry['outputs_text']:
                # Text output
                text_lines = entry['outputs_text'].split('\n')
                if len(text_lines) > 50:
                    text_lines = text_lines[:50] + ['\n... (truncated for PDF) ...']
                truncated_text = '\n'.join(text_lines)
                story.append(Paragraph(f"<pre>{html.escape(truncated_text)}</pre>", code_style))

            # Metadata section
            if entry['metadata']:
                story.append(Spacer(1, 10))
                story.append(Paragraph("Metadata:", styles['Heading3']))
                metadata_text = json.dumps(entry['metadata'], indent=2)
                story.append(Paragraph(f"<pre>{html.escape(metadata_text)}</pre>", code_style))

        # Build PDF
        doc.build(story)

        # Read the file back
        with open(filename, 'rb') as f:
            pdf_data = f.read()

        # Clean up
        os.unlink(filename)

        log_activity(user_id, "export_pdf", "history", f"Exported {len(entries)} {entry_type} entries to PDF")
        add_system_log(f"User {user_id} exported {len(entries)} {entry_type} entries to PDF", "INFO")

        return pdf_data

    except Exception as e:
        print(f"Error creating PDF: {e}")
        add_system_log(f"PDF generation failed: {str(e)}", "ERROR")
        return None

def create_single_entry_pdf(entry, username):
    """
    Create PDF for a single history entry
    """
    return create_pdf_download([entry], entry['type'], entry.get('user_id', 'unknown'), username)

# --- CONVERSATION MANAGEMENT ---

def create_conversation(user_id, title="New Conversation"):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    c.execute(
        "INSERT INTO conversations (user_id, title, created_at, updated_at) VALUES (?, ?, ?, ?)",
        (user_id, title, timestamp, timestamp)
    )
    conversation_id = c.lastrowid
    conn.commit()
    conn.close()
    return conversation_id

def add_message(conversation_id, role, content, model_name, language, response_time=None, tokens_used=0):
    if conversation_id is None:
        raise ValueError("conversation_id cannot be None")

    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    try:
        c.execute(
            """INSERT INTO messages (conversation_id, role, content, model_name, language, timestamp, response_time, tokens_used)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)""",
            (conversation_id, role, content, model_name, language, timestamp, response_time, tokens_used)
        )
        message_id = c.lastrowid

        c.execute(
            "UPDATE conversations SET updated_at = ? WHERE id = ?",
            (timestamp, conversation_id)
        )

        conn.commit()
        return message_id
    except Exception as e:
        conn.rollback()
        raise e
    finally:
        conn.close()
def get_user_conversations(user_id):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute(
        "SELECT id, title, created_at, updated_at, is_shared, share_id FROM conversations WHERE user_id = ? ORDER BY updated_at DESC",
        (user_id,)
    )
    conversations = c.fetchall()
    conn.close()
    return conversations

def get_conversation_messages(conversation_id):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute(
        """SELECT role, content, model_name, language, timestamp, response_time, feedback_score, feedback_comment, tokens_used
        FROM messages WHERE conversation_id = ? ORDER BY timestamp ASC""",
        (conversation_id,)
    )
    messages = c.fetchall()
    conn.close()
    return messages

def update_conversation_title(conversation_id, new_title):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute(
        "UPDATE conversations SET title = ? WHERE id = ?",
        (new_title, conversation_id)
    )
    conn.commit()
    conn.close()

def delete_conversation(conversation_id):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("DELETE FROM messages WHERE conversation_id = ?", (conversation_id,))
    c.execute("DELETE FROM conversations WHERE id = ?", (conversation_id,))
    conn.commit()
    conn.close()

def share_conversation(conversation_id):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    share_id = base64.urlsafe_b64encode(os.urandom(9)).decode('utf-8')
    c.execute(
        "UPDATE conversations SET is_shared=1, share_id=? WHERE id=?",
        (share_id, conversation_id)
    )
    conn.commit()
    conn.close()
    return share_id

def unshare_conversation(conversation_id):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute(
        "UPDATE conversations SET is_shared=0, share_id=NULL WHERE id=?",
        (conversation_id,)
    )
    conn.commit()
    conn.close()

def get_shared_conversation(share_id):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute(
        "SELECT id, title, user_id FROM conversations WHERE share_id=? AND is_shared=1",
        (share_id,)
    )
    conversation = c.fetchone()
    conn.close()
    return conversation

def update_message_feedback(message_id, score, comment):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute(
        "UPDATE messages SET feedback_score = ?, feedback_comment = ? WHERE id = ?",
        (score, comment, message_id)
    )
    conn.commit()
    conn.close()

def log_activity(user_id, activity_type, module, details=""):
    try:
        conn = sqlite3.connect(DB_NAME)
        c = conn.cursor()
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        c.execute(
            "INSERT INTO activity_log (user_id, activity_type, timestamp, module, details) VALUES (?, ?, ?, ?, ?)",
            (user_id, activity_type, timestamp, module, details)
        )
        conn.commit()
    except Exception as e:
        print(f"Error logging activity: {e}")
    finally:
        conn.close()

# --- ANALYTICS FUNCTIONS ---

def get_total_users():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT COUNT(id) FROM users WHERE is_active=1")
    count = c.fetchone()[0]
    conn.close()
    return count

def get_total_queries():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT COUNT(id) FROM messages")
    count = c.fetchone()[0]
    conn.close()
    return count

def get_user_growth_data():
    """Get user growth data with debug information"""
    try:
        conn = sqlite3.connect(DB_NAME)
        c = conn.cursor()

        # DEBUG: Check what's in the users table
        c.execute("SELECT username, created_at, is_active FROM users ORDER BY created_at")
        all_users = c.fetchall()
        print("=== DEBUG: ALL USERS ===")
        for user in all_users:
            print(f"User: {user[0]}, Created: {user[1]}, Active: {user[2]}")

        # The actual growth query
        c.execute("""
            SELECT DATE(created_at) as date, COUNT(*) as count
            FROM users
            WHERE is_active=1
            GROUP BY DATE(created_at)
            ORDER BY date
        """)
        data = c.fetchall()

        print("=== DEBUG: GROWTH DATA ===")
        print(f"Raw growth data: {data}")
        print(f"Number of data points: {len(data)}")

        conn.close()
        return data

    except Exception as e:
        print(f"ERROR in get_user_growth_data: {e}")
        return []

def get_language_usage():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("""
        SELECT language, COUNT(language) as count
        FROM messages
        WHERE language IS NOT NULL
        GROUP BY language
        ORDER BY count DESC
    """)
    data = c.fetchall()
    conn.close()
    return data

def get_module_usage():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("""
        SELECT module, COUNT(*) as count
        FROM activity_log
        WHERE module IS NOT NULL
        GROUP BY module
        ORDER BY count DESC
    """)
    data = c.fetchall()
    conn.close()
    return data

def get_top_users():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("""
        SELECT u.username, COUNT(m.id) as message_count
        FROM users u
        JOIN conversations c ON u.id = c.user_id
        JOIN messages m ON c.id = m.conversation_id
        WHERE u.is_active=1
        GROUP BY u.id, u.username
        ORDER BY message_count DESC
        LIMIT 10
    """)
    data = c.fetchall()
    conn.close()
    return data

def get_feedback_stats():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("""
        SELECT
            AVG(feedback_score) as avg_rating,
            COUNT(*) as total_feedback,
            SUM(CASE WHEN feedback_score >= 4 THEN 1 ELSE 0 END) as positive_feedback
        FROM messages
        WHERE feedback_score > 0
    """)
    data = c.fetchone()
    conn.close()
    return data

def get_global_activity():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("""
        SELECT
            u.username,
            a.activity_type,
            a.module,
            a.timestamp,
            a.details
        FROM activity_log a
        JOIN users u ON a.user_id = u.id
        WHERE u.is_active=1
        ORDER BY a.timestamp DESC
        LIMIT 50
    """)
    data = c.fetchall()
    conn.close()
    return data

# --- 3. AUTHENTICATION (JWT) ---

def create_jwt(username, role, token_type='access'):
    if token_type == 'access':
        secret = JWT_SECRET
        expires = timedelta(hours=24)
    else:
        secret = JWT_REFRESH_SECRET
        expires = timedelta(days=7)

    payload = {
        'sub': username,
        'role': role,
        'type': token_type,
        'iat': datetime.utcnow(),
        'exp': datetime.utcnow() + expires
    }
    token = jwt.encode(payload, secret, algorithm='HS256')
    return token

def verify_jwt(token, token_type='access'):
    try:
        secret = JWT_SECRET if token_type == 'access' else JWT_REFRESH_SECRET
        payload = jwt.decode(token, secret, algorithms=['HS256'])

        if payload.get('type') != token_type:
            return None

        return payload
    except jwt.ExpiredSignatureError:
        st.error("Session expired. Please log in again.")
        return None
    except jwt.InvalidTokenError:
        st.error("Invalid token. Please log in again.")
        return None

# --- 4. MODEL MANAGEMENT WITH GLOBAL CACHE ---

MODELS = {
    "Gemma-2B-IT": "google/gemma-2b-it",
    "DeepSeek-Coder-1.3B": "deepseek-ai/deepseek-coder-1.3b-instruct",
    "Phi-2": "microsoft/phi-2",
    "CodeLlama-7B": "codellama/CodeLlama-7b-hf",
    "StarCoder2-3B": "bigcode/starcoder2-3b"
}

CHAT_MODELS = {
    "Gemma-2B-IT": "google/gemma-2b-it",
    "Phi-2": "microsoft/phi-2"
}

def load_model_to_cache(model_name_key, model_type='code'):
    """Load a model into the global cache"""
    if model_type == 'chat':
        model_path = CHAT_MODELS.get(model_name_key)
        cache_dict = MODEL_CACHE['chat_models']
    else:
        model_path = MODELS.get(model_name_key)
        cache_dict = MODEL_CACHE['code_models']

    if not model_path:
        return None, None

    # Check if already loading
    if model_name_key in MODEL_CACHE['loading_status']:
        return None, None  # Already being loaded

    # Mark as loading
    MODEL_CACHE['loading_status'][model_name_key] = True

    try:
        token = HF_TOKEN
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )

        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True, token=token)

        # Add padding token if missing
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        # Load model
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            quantization_config=quantization_config,
            device_map="auto",
            torch_dtype=torch.bfloat16,
            trust_remote_code=True,
            token=token
        )

        # Store in cache
        cache_dict[model_name_key] = (model, tokenizer)
        MODEL_CACHE['loaded_at'] = datetime.now()

        add_system_log(f"Model {model_name_key} loaded into global cache", "SUCCESS")
        return model, tokenizer

    except Exception as e:
        add_system_log(f"Failed to load model {model_name_key}: {str(e)}", "ERROR")
        # Remove loading status on failure
        if model_name_key in MODEL_CACHE['loading_status']:
            del MODEL_CACHE['loading_status'][model_name_key]
        return None, None

def get_model_from_cache(model_name_key, model_type='code'):
    """Get model from global cache, load if not present"""
    if model_type == 'chat':
        cache_dict = MODEL_CACHE['chat_models']
    else:
        cache_dict = MODEL_CACHE['code_models']

    # Check if model is in cache
    if model_name_key in cache_dict:
        return cache_dict[model_name_key]

    # Check if model is being loaded
    if model_name_key in MODEL_CACHE['loading_status']:
        return None, None  # Still loading

    # Start loading the model
    return load_model_to_cache(model_name_key, model_type)

def preload_essential_models_async():
    """Pre-load essential models in background thread"""
    def load_models():
        essential_models = ["Gemma-2B-IT", "DeepSeek-Coder-1.3B", "Phi-2"]

        for model_name in essential_models:
            if model_name not in MODEL_CACHE['code_models'] and model_name not in MODEL_CACHE['loading_status']:
                load_model_to_cache(model_name, 'code')
                time.sleep(2)  # Small delay between loads

    # Start loading in background thread
    thread = threading.Thread(target=load_models, daemon=True)
    thread.start()

def get_model_status():
    """Get status of all models"""
    status = {}

    for model_name in MODELS.keys():
        if model_name in MODEL_CACHE['code_models']:
            status[model_name] = "✅ Ready"
        elif model_name in MODEL_CACHE['loading_status']:
            status[model_name] = "🔄 Loading..."
        else:
            status[model_name] = "⏳ Not Loaded"

    return status

def generate_code(prompt, model_name_key, language):
    model, tokenizer = get_model_from_cache(model_name_key, 'code')
    if model is None or tokenizer is None:
        return f"Model {model_name_key} is loading. Please wait a moment and try again.", 0

    # Enhanced prompt for better code generation
    enhanced_prompt = f"""
    Generate {language} code for the following request. Provide only the code without explanations:

    Request: {prompt}

    Code:
    """

    inputs = tokenizer(enhanced_prompt, return_tensors="pt").to(model.device)
    attention_mask = inputs.attention_mask

    with torch.no_grad():
        output_ids = model.generate(
            inputs.input_ids,
            attention_mask=attention_mask,
            max_new_tokens=512,
            temperature=0.1,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )

    response_tokens = output_ids[0][inputs.input_ids.shape[-1]:]
    code = tokenizer.decode(response_tokens, skip_special_tokens=True)
    tokens_used = len(response_tokens)

    # Clean up the code
    code = code.strip()
    if "```" in code:
        # Extract code from markdown code blocks
        lines = code.split('\n')
        code_lines = []
        in_code_block = False
        for line in lines:
            if line.strip().startswith('```'):
                in_code_block = not in_code_block
                continue
            if in_code_block or not line.strip().startswith('#'):
                code_lines.append(line)
        code = '\n'.join(code_lines).strip()

    if not code:
        return "Error: Model returned an empty response. Please try again.", 0

    return f"```{language.lower()}\n{code}\n```", tokens_used

def generate_explanation(code, model_name_key, language):
    model, tokenizer = get_model_from_cache(model_name_key, 'code')
    if model is None or tokenizer is None:
        return f"Model {model_name_key} is loading. Please wait a moment and try again.", 0

    prompt = f"""
    Explain the following {language} code in detail. Include:
    1. What the code does
    2. Step-by-step logic explanation
    3. Time complexity analysis
    4. Potential edge cases
    5. Suggestions for improvement

    Code:
    {code}

    Explanation:
    """

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    attention_mask = inputs.attention_mask

    with torch.no_grad():
        output_ids = model.generate(
            inputs.input_ids,
            attention_mask=attention_mask,
            max_new_tokens=512,
            temperature=0.3,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )

    response_tokens = output_ids[0][inputs.input_ids.shape[-1]:]
    explanation = tokenizer.decode(response_tokens, skip_special_tokens=True)
    tokens_used = len(response_tokens)

    return explanation, tokens_used

def chat_with_ai(message, model_name_key):
    model, tokenizer = get_model_from_cache(model_name_key, 'chat')
    if model is None or tokenizer is None:
        return f"Model {model_name_key} is loading. Please wait a moment and try again.", 0

    # Build conversation context
    conversation_history = st.session_state.get('chat_history', [])
    context = "\n".join([f"{'User' if msg['role'] == 'user' else 'Assistant'}: {msg['content']}"
                        for msg in conversation_history[-6:]])  # Last 6 messages for context

    prompt = f"""
    You are CodeGenie, a helpful AI assistant for programming and coding questions.

    Previous conversation:
    {context}

    User: {message}

    Assistant: """

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    attention_mask = inputs.attention_mask

    with torch.no_grad():
        output_ids = model.generate(
            inputs.input_ids,
            attention_mask=attention_mask,
            max_new_tokens=256,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )

    response_tokens = output_ids[0][inputs.input_ids.shape[-1]:]
    response = tokenizer.decode(response_tokens, skip_special_tokens=True)
    tokens_used = len(response_tokens)

    return response, tokens_used

# --- 5. AST VISUALIZER ---

class ASTVisualizer:
    def __init__(self):
        self.graph = graphviz.Digraph(
            comment='Python AST',
            graph_attr={'rankdir': 'TB', 'bgcolor': '#1E1E1E'},
            node_attr={'style': 'filled', 'color': '#569CD6', 'fontcolor': 'white', 'fillcolor': '#2D2D30'},
            edge_attr={'color': '#D4D4D4'}
        )

    def traverse(self, node, parent_id=None):
        node_id = str(id(node))
        node_label = type(node).__name__

        # Enhanced node labeling
        if isinstance(node, ast.FunctionDef):
            node_label = f"Function: {node.name}"
        elif isinstance(node, ast.ClassDef):
            node_label = f"Class: {node.name}"
        elif isinstance(node, ast.Name):
            node_label = f"Variable: {node.id}"
        elif isinstance(node, ast.Call):
            node_label = "Function Call"
        elif isinstance(node, ast.Assign):
            node_label = "Assignment"
        elif isinstance(node, ast.If):
            node_label = "If Statement"
        elif isinstance(node, ast.For):
            node_label = "For Loop"
        elif isinstance(node, ast.While):
            node_label = "While Loop"

        self.graph.node(node_id, label=node_label)

        if parent_id:
            self.graph.edge(parent_id, node_id)

        for field, value in ast.iter_fields(node):
            if isinstance(value, list):
                for item in value:
                    if isinstance(item, ast.AST):
                        self.traverse(item, node_id)
            elif isinstance(value, ast.AST):
                self.traverse(value, node_id)

    def get_graph(self, code):
        try:
            tree = ast.parse(code)
            self.traverse(tree)
            return self.graph
        except SyntaxError as e:
            st.error(f"Python Syntax Error: {e}")
            return None
        except Exception as e:
            st.error(f"Error building AST: {e}")
            return None

# --- 6. EMAIL UTILITIES ---

def send_otp_email(email, otp_code):
    try:
        msg = MIMEMultipart()
        msg['From'] = EMAIL_ADDRESS
        msg['To'] = email
        msg['Subject'] = "CodeGenie - Password Reset OTP"

        body = f"""
        <html>
            <body>
                <h2>CodeGenie Password Reset</h2>
                <p>Your One-Time Password (OTP) for password reset is:</p>
                <h1 style="color: #1A73E8; font-size: 32px; text-align: center;">{otp_code}</h1>
                <p>This OTP will expire in 10 minutes.</p>
                <p>If you didn't request this reset, please ignore this email.</p>
                <br>
                <p>Best regards,<br>CodeGenie Team</p>
            </body>
        </html>
        """

        msg.attach(MIMEText(body, 'html'))

        server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
        server.starttls()
        server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        server.send_message(msg)
        server.quit()

        return True
    except Exception as e:
        print(f"Error sending email: {e}")
        return False

# --- 7. SYSTEM LOGS ---

def add_system_log(message, icon="INFO"):
    if 'system_logs' not in st.session_state:
        st.session_state.system_logs = []
    log_entry = f"[{icon}] {datetime.now().strftime('%H:%M:%S')} - {message}"
    st.session_state.system_logs.insert(0, log_entry)
    st.session_state.system_logs = st.session_state.system_logs[:50]

# --- 8. HISTORY PAGES ---

def show_generator_history_page():
    apply_theme()
    st.title("Code Generator History")

    if not st.session_state.logged_in:
        st.error("Please log in to view your history.")
        return

    user_id = st.session_state.user_id

    # Search and filters
    col1, col2, col3, col4 = st.columns([2, 1, 1, 1])

    with col1:
        search_query = st.text_input("Search history", placeholder="Search prompts or generated code...")

    with col2:
        time_filter = st.selectbox("Time filter", ["All", "24h", "7d", "30d"])

    with col3:
        page_size = st.selectbox("Page size", [10, 20, 50], index=1)

    with col4:
        st.write("")  # Spacer
        if st.button("Refresh"):
            st.rerun()

    # Get current page from session state
    if 'gen_history_page' not in st.session_state:
        st.session_state.gen_history_page = 1

    # Get history data
    history_data = get_user_history_entries(
        user_id,
        'generator',
        page=st.session_state.gen_history_page,
        page_size=page_size,
        search_query=search_query if search_query else None,
        time_filter=time_filter if time_filter != "All" else None
    )

    entries = history_data['entries']
    total_pages = history_data['total_pages']

    # Display entries
    if not entries:
        st.info("No generator history found. Your code generation activities will appear here.")
    else:
        st.write(f"Showing {len(entries)} of {history_data['total_count']} entries")

        # Bulk download button
        if st.button("📥 Download All as PDF", type="secondary"):
            pdf_data = create_pdf_download(entries, 'generator', user_id, st.session_state.username)
            if pdf_data:
                st.download_button(
                    label="⬇️ Download PDF Now",
                    data=pdf_data,
                    file_name=f"generator-history-{user_id}-{datetime.now().strftime('%Y%m%d')}.pdf",
                    mime="application/pdf",
                    use_container_width=True
                )
            else:
                st.error("Failed to generate PDF. Please try again.")

        st.markdown("---")

        # Display entries in a table-like format
        for i, entry in enumerate(entries):
            with st.expander(f"🛠️ {entry['created_at'][:16]} - {entry['inputs'].get('prompt', 'No prompt')[:80]}...", expanded=False):
                col1, col2 = st.columns([3, 1])

                with col1:
                    st.write(f"**Prompt:** {entry['inputs'].get('prompt', 'No prompt')}")
                    st.write(f"**Language:** {entry['inputs'].get('language', 'Unknown')}")
                    st.write(f"**Model:** {entry['metadata'].get('model', 'Unknown')}")

                    if entry['outputs_code']:
                        st.code(entry['outputs_code'], language=entry['inputs'].get('language', '').lower())

                with col2:
                    st.write(f"**Created:** {entry['created_at'][:16]}")

                    # Single entry download
                    if st.button("📄 PDF", key=f"pdf_{entry['id']}", use_container_width=True):
                        pdf_data = create_single_entry_pdf(entry, st.session_state.username)
                        if pdf_data:
                            st.download_button(
                                label="⬇️ Download PDF",
                                data=pdf_data,
                                file_name=f"generator-entry-{entry['id'][:8]}.pdf",
                                mime="application/pdf",
                                use_container_width=True,
                                key=f"dl_{entry['id']}"
                            )
                        else:
                            st.error("Failed to generate PDF")

                    # Delete button
                    if st.button("🗑️ Delete", key=f"del_{entry['id']}", use_container_width=True):
                        if delete_history_entry(entry['id'], user_id):
                            st.success("Entry deleted")
                            st.rerun()
                        else:
                            st.error("Failed to delete entry")

        # Pagination
        if total_pages > 1:
            st.markdown("---")
            col_prev, col_info, col_next = st.columns([1, 2, 1])

            with col_prev:
                if st.session_state.gen_history_page > 1:
                    if st.button("← Previous"):
                        st.session_state.gen_history_page -= 1
                        st.rerun()

            with col_info:
                st.write(f"Page {st.session_state.gen_history_page} of {total_pages}")

            with col_next:
                if st.session_state.gen_history_page < total_pages:
                    if st.button("Next →"):
                        st.session_state.gen_history_page += 1
                        st.rerun()

def show_explainer_history_page():
    apply_theme()
    st.title("Code Explainer History")

    if not st.session_state.logged_in:
        st.error("Please log in to view your history.")
        return

    user_id = st.session_state.user_id

    # Search and filters
    col1, col2, col3, col4 = st.columns([2, 1, 1, 1])

    with col1:
        search_query = st.text_input("Search history", placeholder="Search code or explanations...", key="explainer_search")

    with col2:
        time_filter = st.selectbox("Time filter", ["All", "24h", "7d", "30d"], key="explainer_time")

    with col3:
        page_size = st.selectbox("Page size", [10, 20, 50], index=1, key="explainer_size")

    with col4:
        st.write("")  # Spacer
        if st.button("Refresh", key="explainer_refresh"):
            st.rerun()

    # Get current page from session state
    if 'exp_history_page' not in st.session_state:
        st.session_state.exp_history_page = 1

    # Get history data
    history_data = get_user_history_entries(
        user_id,
        'explainer',
        page=st.session_state.exp_history_page,
        page_size=page_size,
        search_query=search_query if search_query else None,
        time_filter=time_filter if time_filter != "All" else None
    )

    entries = history_data['entries']
    total_pages = history_data['total_pages']

    # Display entries
    if not entries:
        st.info("No explainer history found. Your code explanation activities will appear here.")
    else:
        st.write(f"Showing {len(entries)} of {history_data['total_count']} entries")

        # Bulk download button
        if st.button("📥 Download All as PDF", type="secondary", key="exp_bulk_dl"):
            pdf_data = create_pdf_download(entries, 'explainer', user_id, st.session_state.username)
            if pdf_data:
                st.download_button(
                    label="⬇️ Download PDF Now",
                    data=pdf_data,
                    file_name=f"explainer-history-{user_id}-{datetime.now().strftime('%Y%m%d')}.pdf",
                    mime="application/pdf",
                    use_container_width=True,
                    key="exp_dl_now"
                )
            else:
                st.error("Failed to generate PDF. Please try again.")

        st.markdown("---")

        # Display entries in a table-like format
        for i, entry in enumerate(entries):
            with st.expander(f"📖 {entry['created_at'][:16]} - Code Explanation", expanded=False):
                col1, col2 = st.columns([3, 1])

                with col1:
                    st.write(f"**Language:** {entry['inputs'].get('language', 'Unknown')}")
                    st.write(f"**Model:** {entry['metadata'].get('model', 'Unknown')}")

                    # Show input code snippet
                    if entry['inputs'].get('code'):
                        st.write("**Input Code:**")
                        st.code(entry['inputs'].get('code')[:500] + "..." if len(entry['inputs'].get('code', '')) > 500 else entry['inputs'].get('code', ''),
                               language=entry['inputs'].get('language', '').lower())

                    # Show explanation
                    if entry['outputs_text']:
                        st.write("**Explanation:**")
                        st.write(entry['outputs_text'][:1000] + "..." if len(entry['outputs_text']) > 1000 else entry['outputs_text'])

                with col2:
                    st.write(f"**Created:** {entry['created_at'][:16]}")

                    # Single entry download
                    if st.button("📄 PDF", key=f"exp_pdf_{entry['id']}", use_container_width=True):
                        pdf_data = create_single_entry_pdf(entry, st.session_state.username)
                        if pdf_data:
                            st.download_button(
                                label="⬇️ Download PDF",
                                data=pdf_data,
                                file_name=f"explainer-entry-{entry['id'][:8]}.pdf",
                                mime="application/pdf",
                                use_container_width=True,
                                key=f"exp_dl_{entry['id']}"
                            )
                        else:
                            st.error("Failed to generate PDF")

                    # Delete button
                    if st.button("🗑️ Delete", key=f"exp_del_{entry['id']}", use_container_width=True):
                        if delete_history_entry(entry['id'], user_id):
                            st.success("Entry deleted")
                            st.rerun()
                        else:
                            st.error("Failed to delete entry")

        # Pagination
        if total_pages > 1:
            st.markdown("---")
            col_prev, col_info, col_next = st.columns([1, 2, 1])

            with col_prev:
                if st.session_state.exp_history_page > 1:
                    if st.button("← Previous", key="exp_prev"):
                        st.session_state.exp_history_page -= 1
                        st.rerun()

            with col_info:
                st.write(f"Page {st.session_state.exp_history_page} of {total_pages}")

            with col_next:
                if st.session_state.exp_history_page < total_pages:
                    if st.button("Next →", key="exp_next"):
                        st.session_state.exp_history_page += 1
                        st.rerun()

# --- 9. STREAMLIT APP UI ---

def initialize_state():
    # Authentication state
    if "logged_in" not in st.session_state: st.session_state.logged_in = False
    if "username" not in st.session_state: st.session_state.username = None
    if "role" not in st.session_state: st.session_state.role = None
    if "user_id" not in st.session_state: st.session_state.user_id = None
    if "email" not in st.session_state: st.session_state.email = None
    if "page" not in st.session_state: st.session_state.page = "Login"
    if "token" not in st.session_state: st.session_state.token = None
    if "refresh_token" not in st.session_state: st.session_state.refresh_token = None

    # Conversation state
    if "current_conversation_id" not in st.session_state: st.session_state.current_conversation_id = None
    if "conversation_messages" not in st.session_state: st.session_state.conversation_messages = []
    if "conversations" not in st.session_state: st.session_state.conversations = []
    if "chat_history" not in st.session_state: st.session_state.chat_history = []

    # UI state
    if "theme" not in st.session_state: st.session_state.theme = 'dark'
    if "last_output" not in st.session_state: st.session_state.last_output = ""
    if "reset_step" not in st.session_state: st.session_state.reset_step = 1
    if "reset_user" not in st.session_state: st.session_state.reset_user = None
    if "reset_email" not in st.session_state: st.session_state.reset_email = None

    # Model state
    if 'loaded_code_model_name' not in st.session_state: st.session_state.loaded_code_model_name = None
    if 'loaded_code_model' not in st.session_state: st.session_state.loaded_code_model = None
    if 'loaded_code_tokenizer' not in st.session_state: st.session_state.loaded_code_tokenizer = None
    if 'loaded_chat_model_name' not in st.session_state: st.session_state.loaded_chat_model_name = None
    if 'loaded_chat_model' not in st.session_state: st.session_state.loaded_chat_model = None
    if 'loaded_chat_tokenizer' not in st.session_state: st.session_state.loaded_chat_tokenizer = None

    # Admin state
    if 'admin_boot_animation_done' not in st.session_state: st.session_state.admin_boot_animation_done = False
    if 'gpu_history' not in st.session_state: st.session_state.gpu_history = []
    if 'system_logs' not in st.session_state: st.session_state.system_logs = []

    # Chat state
    if 'chat_conversation_id' not in st.session_state: st.session_state.chat_conversation_id = None
    if 'generated_share_url' not in st.session_state: st.session_state.generated_share_url = None
    if 'generated_share_id' not in st.session_state: st.session_state.generated_share_id = None

    # History state
    if 'gen_history_page' not in st.session_state: st.session_state.gen_history_page = 1
    if 'exp_history_page' not in st.session_state: st.session_state.exp_history_page = 1

    # Pre-loading state
    if 'models_preloaded' not in st.session_state: st.session_state.models_preloaded = False

    # Feedback state
    if 'feedback_rating' not in st.session_state: st.session_state.feedback_rating = 0

def apply_theme():
    theme = st.session_state.get('theme', 'dark')
    if theme == 'dark':
        css = """
        <style>
        .stApp {
            background-color: #0f0f23;
            color: #e4e4e7;
            font-family: 'Inter', -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;
        }
        .stButton>button {
            background-color: #6366f1;
            color: white;
            border-radius: 8px;
            border: none;
            padding: 0.625rem 1.25rem;
            font-weight: 500;
            font-size: 0.875rem;
            transition: all 0.2s ease;
        }
        .stButton>button:hover {
            background-color: #4f46e5;
            box-shadow: 0 4px 12px rgba(99, 102, 241, 0.4);
        }
        .stButton>button[kind="secondary"] {
            background-color: #27272a;
            color: #e4e4e7;
            border: 1px solid #3f3f46;
        }
        .stButton>button[kind="secondary"]:hover {
            background-color: #3f3f46;
        }
        .stTextInput>div>div>input {
            background-color: #18181b;
            color: #e4e4e7;
            border-radius: 8px;
            border: 1px solid #3f3f46;
            padding: 0.625rem 0.875rem;
        }
        .stTextInput>div>div>input:focus {
            border-color: #6366f1;
            box-shadow: 0 0 0 3px rgba(99, 102, 241, 0.1);
        }
        .stTextArea>div>div>textarea {
            background-color: #18181b;
            color: #e4e4e7;
            border-radius: 8px;
            border: 1px solid #3f3f46;
            padding: 0.625rem 0.875rem;
        }
        .stTextArea>div>div>textarea:focus {
            border-color: #6366f1;
            box-shadow: 0 0 0 3px rgba(99, 102, 241, 0.1);
        }
        .stSelectbox>div>div>select {
            background-color: #18181b;
            color: #e4e4e7;
            border-radius: 8px;
            border: 1px solid #3f3f46;
            padding: 0.625rem 0.875rem;
        }
        h1 {
            font-weight: 700;
            font-size: 2rem;
            letter-spacing: -0.025em;
            color: #fafafa;
        }
        h2 {
            font-weight: 600;
            font-size: 1.5rem;
            color: #f4f4f5;
        }
        h3 {
            font-weight: 600;
            font-size: 1.25rem;
            color: #e4e4e7;
        }
        .conversation-item { padding: 10px; margin: 5px 0; border-radius: 8px; cursor: pointer; transition: all 0.3s; }
        .conversation-item:hover { background-color: #27272a; }
        .conversation-item.active { background-color: #6366f1; color: white; }
        .chat-message { padding: 16px; border-radius: 12px; margin: 8px 0; }
        .user-message { background-color: #6366f1; color: white; }
        .assistant-message { background-color: #18181b; color: #e4e4e7; border: 1px solid #3f3f46; }
        </style>
        """
    else:
        css = """
        <style>
        .stApp {
            background-color: #fafafa;
            color: #18181b;
            font-family: 'Inter', -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;
        }
        .stButton>button {
            background-color: #6366f1;
            color: white;
            border-radius: 8px;
            border: none;
            padding: 0.625rem 1.25rem;
            font-weight: 500;
            font-size: 0.875rem;
            transition: all 0.2s ease;
        }
        .stButton>button:hover {
            background-color: #4f46e5;
            box-shadow: 0 4px 12px rgba(99, 102, 241, 0.3);
        }
        .stButton>button[kind="secondary"] {
            background-color: #f4f4f5;
            color: #18181b;
            border: 1px solid #e4e4e7;
        }
        .stButton>button[kind="secondary"]:hover {
            background-color: #e4e4e7;
        }
        .stTextInput>div>div>input {
            background-color: #FFFFFF;
            color: #18181b;
            border-radius: 8px;
            border: 1px solid #e4e4e7;
            padding: 0.625rem 0.875rem;
        }
        .stTextInput>div>div>input:focus {
            border-color: #6366f1;
            box-shadow: 0 0 0 3px rgba(99, 102, 241, 0.1);
        }
        .stTextArea>div>div>textarea {
            background-color: #FFFFFF;
            color: #18181b;
            border-radius: 8px;
            border: 1px solid #e4e4e7;
            padding: 0.625rem 0.875rem;
        }
        .stTextArea>div>div>textarea:focus {
            border-color: #6366f1;
            box-shadow: 0 0 0 3px rgba(99, 102, 241, 0.1);
        }
        .stSelectbox>div>div>select {
            background-color: #FFFFFF;
            color: #18181b;
            border-radius: 8px;
            border: 1px solid #e4e4e7;
            padding: 0.625rem 0.875rem;
        }
        h1 {
            font-weight: 700;
            font-size: 2rem;
            letter-spacing: -0.025em;
            color: #09090b;
        }
        h2 {
            font-weight: 600;
            font-size: 1.5rem;
            color: #18181b;
        }
        h3 {
            font-weight: 600;
            font-size: 1.25rem;
            color: #27272a;
        }
        .conversation-item { padding: 10px; margin: 5px 0; border-radius: 8px; cursor: pointer; transition: all 0.3s; }
        .conversation-item:hover { background-color: #f4f4f5; }
        .conversation-item.active { background-color: #6366f1; color: white; }
        .chat-message { padding: 16px; border-radius: 12px; margin: 8px 0; }
        .user-message { background-color: #6366f1; color: white; }
        .assistant-message { background-color: #FFFFFF; color: #18181b; border: 1px solid #e4e4e7; box-shadow: 0 1px 3px rgba(0, 0, 0, 0.1); }
        </style>
        """
    st.markdown(css, unsafe_allow_html=True)

# --- STAR RATING COMPONENT ---
def star_rating_component(label, key):
    """
    Create a star rating component with emojis
    """
    st.markdown(f"**{label}**")

    # Initialize rating in session state if not exists
    if f"{key}_rating" not in st.session_state:
        st.session_state[f"{key}_rating"] = 0

    # Create columns for stars
    cols = st.columns(5)
    rating = st.session_state[f"{key}_rating"]

    # Star emojis (filled and empty)
    filled_star = "⭐"
    empty_star = "☆"

    # Display stars and handle clicks
    for i in range(5):
        with cols[i]:
            star_value = i + 1
            if st.button(filled_star if star_value <= rating else empty_star,
                        key=f"{key}_{star_value}",
                        use_container_width=True):
                st.session_state[f"{key}_rating"] = star_value
                st.rerun()

    # Display current rating text
    if rating > 0:
        rating_texts = {
            1: "Poor ⭐",
            2: "Fair ⭐⭐",
            3: "Good ⭐⭐⭐",
            4: "Very Good ⭐⭐⭐⭐",
            5: "Excellent ⭐⭐⭐⭐⭐"
        }
        st.caption(f"Selected: {rating_texts[rating]}")

    return rating

# --- AUTHENTICATION PAGES ---

def show_login_page(is_first_user, admin_count):
    apply_theme()

    st.markdown("""
        <style>
        .login-header {
            text-align: center;
            padding: 2rem 0;
        }
        .login-header h1 {
            font-size: 3rem;
            margin-bottom: 0.5rem;
            background: linear-gradient(135deg, #6366f1, #8b5cf6);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            font-weight: 700;
            letter-spacing: -0.05em;
        }
        </style>
    """, unsafe_allow_html=True)

    col1, col2 = st.columns([1, 1])

    with col1:
        st.markdown('<div class="login-header"><h1>CodeGenie</h1><p>AI-Powered Code Generation & Explanation</p></div>', unsafe_allow_html=True)

        st.markdown("""
        ### Features
        - **AI Code Generation** - Generate code in multiple languages
        - **Code Explanation** - Understand code with detailed explanations
        - **AI Chat Assistant** - Get help with programming questions
        - **AST Visualization** - Visualize code structure
        - **History & Export** - Track and download your work
        - **Admin Dashboard** - Advanced analytics & user management

        ### Performance
        - **Global Model Cache** - Models loaded once, used by all users
        - **Instant Access** - No waiting for model loading
        - **Background Loading** - Essential models pre-loaded automatically
        """)

    with col2:
        with st.container(border=True):
            tab1, tab2 = st.tabs(["Login", "Sign Up"])

            with tab1:
                with st.form("login_form"):
                    st.subheader("Welcome Back")
                    username = st.text_input("Username", placeholder="Enter your username")
                    password = st.text_input("Password", type="password", placeholder="Enter your password")

                    col_a, col_b = st.columns([2, 1])
                    with col_a:
                        login_button = st.form_submit_button("Login →", use_container_width=True)
                    with col_b:
                        if st.form_submit_button("Forgot Password?", use_container_width=True):
                            st.session_state.page = "Forgot Password"
                            st.rerun()

                    if login_button:
                        if not username or not password:
                            st.error("Please fill in all fields")
                        else:
                            user_data = verify_user(username, password)
                            if user_data:
                                st.session_state.logged_in = True
                                st.session_state.username = user_data["username"]
                                st.session_state.role = user_data["role"]
                                st.session_state.user_id = user_data["user_id"]
                                st.session_state.email = user_data["email"]
                                st.session_state.token = create_jwt(user_data["username"], user_data["role"])
                                st.session_state.refresh_token = create_jwt(user_data["username"], user_data["role"], 'refresh')

                                # Load user conversations
                                user_id = get_user_id(user_data["username"])
                                st.session_state.conversations = get_user_conversations(user_id)

                                # Create first conversation if none exists
                                if not st.session_state.conversations:
                                    new_conv_id = create_conversation(user_id, "Welcome Conversation")
                                    st.session_state.current_conversation_id = new_conv_id
                                    st.session_state.conversation_messages = []
                                    st.session_state.conversations = get_user_conversations(user_id)
                                else:
                                    st.session_state.current_conversation_id = st.session_state.conversations[0][0]
                                    st.session_state.conversation_messages = get_conversation_messages(st.session_state.current_conversation_id)

                                # Log activity
                                log_activity(user_data["user_id"], "login", "authentication")
                                add_system_log(f"User '{user_data['username']}' logged in.", "INFO")

                                # If admin, ensure models are loaded
                                if user_data["role"] == 'admin':
                                    # Start background loading of essential models
                                    preload_essential_models_async()
                                    st.success("🔧 Admin detected - Pre-loading essential models for all users...")

                                # Redirect based on role
                                if user_data["role"] == 'admin':
                                    st.session_state.page = "Dashboard"
                                else:
                                    st.session_state.page = "Generator"
                                st.rerun()
                            else:
                                st.error("Invalid username or password")

            with tab2:
                with st.form("signup_form"):
                    st.subheader("Create Account")

                    new_username = st.text_input("Username*", placeholder="Choose a username")
                    email = st.text_input("Email*", placeholder="your.email@example.com")
                    new_password = st.text_input("Password*", type="password", placeholder="Create a strong password")
                    new_password_confirm = st.text_input("Confirm Password*", type="password", placeholder="Confirm your password")

                    st.markdown("---")
                    st.markdown("**Security Settings**")
                    question = st.selectbox("Security Question*", [
                        "What was the name of your first pet?",
                        "What is your mother's maiden name?",
                        "What city were you born in?",
                        "What was your first car?",
                        "What is your favorite book?"
                    ])
                    answer = st.text_input("Your Answer*", type="password", placeholder="Answer for security verification")

                    if is_first_user:
                        st.info("You're the first user! You'll be registered as an **Admin**.")
                        role = 'admin'
                    else:
                        role = 'user'

                    signup_button = st.form_submit_button("Create Account →", use_container_width=True)

                    if signup_button:
                        if not all([new_username, email, new_password, new_password_confirm, answer]):
                            st.error("Please fill out all required fields.")
                        elif new_password != new_password_confirm:
                            st.error("Passwords do not match.")
                        elif len(new_password) < 6:
                            st.error("Password must be at least 6 characters long.")
                        else:
                            success, message = add_user(new_username, email, new_password, question, answer, role)
                            if success:
                                add_system_log(f"New user '{new_username}' registered as '{role}'.", "INFO")
                                st.success(f"Account created successfully! You are now a {role}. Please log in.")
                                st.session_state.page = "Login"
                                st.rerun()
                            else:
                                st.error(message)

def show_forgot_password_page():
    apply_theme()

    st.title("Password Recovery")
    st.markdown("Recover your account using email OTP or security questions")

    with st.container(border=True):
        if st.session_state.reset_step == 1:
            st.subheader("Step 1: Enter Your Email")
            with st.form("forgot_password_email"):
                email = st.text_input("Registered Email Address", placeholder="your.email@example.com")
                submit_email = st.form_submit_button("Send OTP →", use_container_width=True)

                if submit_email:
                    if email:
                        user = get_user_by_email(email)
                        if user:
                            # Generate OTP
                            otp_code = str(random.randint(100000, 999999))
                            store_otp(email, otp_code)

                            # Send email
                            if send_otp_email(email, otp_code):
                                st.session_state.reset_email = email
                                st.session_state.reset_step = 2
                                st.success(f"OTP sent to {email}")
                                st.rerun()
                            else:
                                st.error("Failed to send OTP email. Please try the security question method.")
                        else:
                            st.error("Email not found in our system.")
                    else:
                        st.error("Please enter your email address.")

            st.markdown("---")
            st.subheader("Alternative: Security Question")
            if st.button("Use Security Question Instead", use_container_width=True):
                st.session_state.reset_step = 4
                st.rerun()

        elif st.session_state.reset_step == 2:
            st.subheader("Step 2: Enter OTP")
            st.info(f"OTP sent to: {st.session_state.reset_email}")

            with st.form("forgot_password_otp"):
                otp = st.text_input("6-digit OTP", placeholder="Enter OTP from email")
                submit_otp = st.form_submit_button("Verify OTP →", use_container_width=True)

                if submit_otp:
                    if otp and len(otp) == 6:
                        if verify_otp(st.session_state.reset_email, otp):
                            st.session_state.reset_step = 3
                            st.success("OTP verified successfully!")
                            st.rerun()
                        else:
                            st.error("Invalid or expired OTP.")
                    else:
                        st.error("Please enter a valid 6-digit OTP.")

            if st.button("↩ Back", use_container_width=True):
                st.session_state.reset_step = 1
                st.rerun()

        elif st.session_state.reset_step == 3:
            st.subheader("Step 3: Set New Password")
            st.success("OTP verified! Now set your new password.")

            with st.form("reset_password"):
                new_password = st.text_input("New Password", type="password")
                confirm_password = st.text_input("Confirm New Password", type="password")
                submit_reset = st.form_submit_button("Reset Password →", use_container_width=True)

                if submit_reset:
                    if new_password and confirm_password:
                        if new_password == confirm_password:
                            # Get username from email
                            user = get_user_by_email(st.session_state.reset_email)
                            if user:
                                username = user[0]
                                update_user_password(username, new_password)
                                add_system_log(f"User '{username}' reset password via OTP.", "INFO")
                                st.success("Password reset successfully! You can now log in.")
                                st.session_state.reset_step = 1
                                st.session_state.page = "Login"
                                st.rerun()
                            else:
                                st.error("User not found.")
                        else:
                            st.error("Passwords do not match.")
                    else:
                        st.error("Please fill in both password fields.")

        elif st.session_state.reset_step == 4:
            st.subheader("Security Question Verification")
            with st.form("security_question"):
                username = st.text_input("Username", placeholder="Enter your username")
                submit_user = st.form_submit_button("Next →", use_container_width=True)

                if submit_user:
                    if username:
                        user_details = get_user_details(username)
                        if user_details:
                            st.session_state.reset_user = user_details
                            st.session_state.reset_step = 5
                            st.rerun()
                        else:
                            st.error("Username not found.")
                    else:
                        st.error("Please enter your username.")

        elif st.session_state.reset_step == 5:
            st.subheader("Answer Security Question")
            st.info(f"**Question:** {st.session_state.reset_user['question']}")

            with st.form("security_answer"):
                answer = st.text_input("Your Answer", type="password")
                submit_answer = st.form_submit_button("Verify Answer →", use_container_width=True)

                if submit_answer:
                    if answer:
                        if check_hash(answer, st.session_state.reset_user['answer_hash']):
                            st.session_state.reset_step = 6
                            st.rerun()
                        else:
                            st.error("Incorrect answer.")
                    else:
                        st.error("Please enter your answer.")

        elif st.session_state.reset_step == 6:
            st.subheader("Set New Password")
            st.success("Security question verified! Now set your new password.")

            with st.form("reset_password_security"):
                new_password = st.text_input("New Password", type="password")
                confirm_password = st.text_input("Confirm New Password", type="password")
                submit_reset = st.form_submit_button("Reset Password →", use_container_width=True)

                if submit_reset:
                    if new_password and confirm_password:
                        if new_password == confirm_password:
                            update_user_password(st.session_state.reset_user['username'], new_password)
                            add_system_log(f"User '{st.session_state.reset_user['username']}' reset password via security question.", "INFO")
                            st.success("Password reset successfully! You can now log in.")
                            st.session_state.reset_step = 1
                            st.session_state.page = "Login"
                            st.rerun()
                        else:
                            st.error("Passwords do not match.")
                    else:
                        st.error("Please fill in both password fields.")

    if st.button("← Back to Login", use_container_width=True):
        st.session_state.page = "Login"
        st.session_state.reset_step = 1
        st.rerun()

# --- USER PAGES ---

def show_generator_page():
    apply_theme()

    # Sidebar for conversations
    with st.sidebar:
        st.subheader("Conversations")

        # New conversation button
        if st.button("New Chat", use_container_width=True, type="primary"):
            user_id = get_user_id(st.session_state.username)
            new_conv_id = create_conversation(user_id, "New Conversation")
            st.session_state.current_conversation_id = new_conv_id
            st.session_state.conversation_messages = []
            st.session_state.conversations = get_user_conversations(user_id)
            st.rerun()

        st.markdown("---")

        # Conversation list
        for conv_id, title, created_at, updated_at, is_shared, share_id in st.session_state.conversations:
            is_active = conv_id == st.session_state.current_conversation_id
            prefix = "[Shared] " if is_shared else ""
            button_label = f"{prefix}{title}"

            col1, col2 = st.columns([3, 1])
            with col1:
                if st.button(button_label, key=f"conv_{conv_id}", use_container_width=True,
                            type="primary" if is_active else "secondary"):
                    st.session_state.current_conversation_id = conv_id
                    st.session_state.conversation_messages = get_conversation_messages(conv_id)
                    st.rerun()

            with col2:
                if is_shared:
                    st.button("Link", key=f"share_{conv_id}", help="Shared conversation",
                             use_container_width=True)

    # Main content area
    st.title("Code Generator")

    # Model selection with performance info
    col1, col2 = st.columns([2, 1])
    with col1:
        # Show models with status
        model_options = ["Gemma-2B-IT", "DeepSeek-Coder-1.3B", "Phi-2", "CodeLlama-7B", "StarCoder2-3B"]
        selected_model = st.selectbox(
            "Select AI Model",
            model_options,
            key='gen_model_key',
            help="Check model status in sidebar"
        )

    with col2:
        language = st.selectbox(
            "Language",
            ["Python", "JavaScript", "Java", "C++", "SQL", "HTML", "CSS", "Go", "Rust"],
            key="gen_language"
        )

    # Performance tips
    with st.expander("💡 Performance Tips"):
        st.markdown("""
        - **Global Model Cache**: Models are shared across all users
        - **Instant Access**: No waiting for model loading
        - **Admin Pre-loading**: Essential models loaded automatically by admin users
        - **Fast Models**: Gemma-2B-IT and DeepSeek-Coder-1.3B are fastest
        """)

    # Chat interface
    chat_container = st.container(height=500, border=True)
    with chat_container:
        for role, content, model_name, msg_language, timestamp, response_time, feedback_score, feedback_comment, tokens_used in st.session_state.conversation_messages:
            if role == "user":
                with st.chat_message("user"):
                    st.markdown(f"**You:** {content}")
            else:
                with st.chat_message("assistant"):
                    st.markdown(content)
                    with st.expander("Details"):
                        st.write(f"**Model:** {model_name}")
                        st.write(f"**Language:** {msg_language}")
                        st.write(f"**Time:** {timestamp}")
                        if response_time:
                            st.write(f"**Response Time:** {response_time:.2f}s")
                        if tokens_used:
                            st.write(f"**Tokens Used:** {tokens_used}")
                        if feedback_score:
                            stars = "⭐" * feedback_score + "☆" * (5 - feedback_score)
                            st.write(f"**Feedback:** {stars} ({feedback_score}/5)")
                            if feedback_comment:
                                st.write(f"**Comment:** {feedback_comment}")

    # Input area
    prompt = st.chat_input("Describe the code you want to generate...")

    if prompt:
        # Add user message
        user_id = get_user_id(st.session_state.username)
        add_message(st.session_state.current_conversation_id, "user", prompt, None, language)

        # Update conversation title if first message
        current_messages = get_conversation_messages(st.session_state.current_conversation_id)
        if len(current_messages) == 1:
            new_title = prompt[:30] + "..." if len(prompt) > 30 else prompt
            update_conversation_title(st.session_state.current_conversation_id, new_title)
            st.session_state.conversations = get_user_conversations(user_id)

        # Generate code
        with st.spinner(f"Generating {language} code with {selected_model}..."):
            start_time = time.time()
            try:
                response, tokens_used = generate_code(prompt, selected_model, language)
                end_time = time.time()
                latency = end_time - start_time
            except Exception as e:
                response = f"Error: {str(e)}"
                tokens_used = 0
                latency = 0

        # Add assistant message
        message_id = add_message(
            st.session_state.current_conversation_id,
            "assistant",
            response,
            selected_model,
            language,
            latency,
            tokens_used
        )

        # Create history entry for generator
        history_inputs = {
            "prompt": prompt,
            "language": language,
            "model": selected_model
        }
        history_metadata = {
            "model": selected_model,
            "latency_ms": round(latency * 1000, 2),
            "tokens_used": tokens_used,
            "conversation_id": st.session_state.current_conversation_id
        }

        create_history_entry(
            user_id=user_id,
            entry_type='generator',
            inputs=history_inputs,
            outputs_code=response,
            metadata=history_metadata
        )

        # Log activity
        log_activity(st.session_state.user_id, "code_generation", "generator",
                    f"Generated {language} code with {selected_model}")

        st.session_state.last_message_id = message_id
        st.session_state.conversation_messages = get_conversation_messages(st.session_state.current_conversation_id)
        st.rerun()

    # Action buttons
    col1, col2, col3, col4 = st.columns(4)

    with col1:
        if st.button("New Chat", use_container_width=True):
            user_id = get_user_id(st.session_state.username)
            new_conv_id = create_conversation(user_id, "New Conversation")
            st.session_state.current_conversation_id = new_conv_id
            st.session_state.conversation_messages = []
            st.session_state.conversations = get_user_conversations(user_id)
            st.rerun()

    with col2:
        if st.button("Copy Last", use_container_width=True):
            if st.session_state.conversation_messages:
                last_msg = st.session_state.conversation_messages[-1]
                if last_msg[0] == "assistant":
                    st.code(last_msg[1], language=language.lower())
                    st.success("Output displayed above. Please copy manually.")

    with col3:
        if st.session_state.conversation_messages:
            if st.button("Share", use_container_width=True):
                share_id = share_conversation(st.session_state.current_conversation_id)
                st.success(f"Conversation shared! Share ID: {share_id}")

    with col4:
        if st.button("Delete", use_container_width=True):
            delete_conversation(st.session_state.current_conversation_id)
            user_id = get_user_id(st.session_state.username)
            st.session_state.conversations = get_user_conversations(user_id)
            if st.session_state.conversations:
                st.session_state.current_conversation_id = st.session_state.conversations[0][0]
                st.session_state.conversation_messages = get_conversation_messages(st.session_state.current_conversation_id)
            else:
                new_conv_id = create_conversation(user_id, "New Conversation")
                st.session_state.current_conversation_id = new_conv_id
                st.session_state.conversation_messages = []
            st.rerun()

    # Feedback system with star ratings
    if hasattr(st.session_state, 'last_message_id'):
        st.markdown("---")
        st.subheader("🌟 Rate This Response")

        # Use radio buttons for star rating (compatible with forms)
        rating = st.radio(
            "How would you rate this code generation?",
            options=[1, 2, 3, 4, 5],
            format_func=lambda x: f"{'⭐' * x} ({x}/5)",
            horizontal=True,
            key="gen_feedback_rating"
        )

        comment = st.text_area("Additional comments (optional):", placeholder="Any specific feedback or suggestions...", key="gen_feedback_comment")

        # Submit button outside the form context
        if st.button("Submit Feedback", use_container_width=True, key="gen_feedback_submit"):
            if rating > 0:
                update_message_feedback(st.session_state.last_message_id, rating, comment)

                # Show thank you message with the rating
                rating_texts = {
                    1: "Poor ⭐",
                    2: "Fair ⭐⭐",
                    3: "Good ⭐⭐⭐",
                    4: "Very Good ⭐⭐⭐⭐",
                    5: "Excellent ⭐⭐⭐⭐⭐"
                }
                st.success(f"Thank you for your {rating_texts[rating]} feedback!")

                del st.session_state.last_message_id
                st.session_state.conversation_messages = get_conversation_messages(st.session_state.current_conversation_id)
                st.rerun()
            else:
                st.warning("Please select a star rating before submitting feedback.")

def show_explainer_page():
    apply_theme()
    st.title("Code Explainer")

    col1, col2 = st.columns([1, 1])

    with col1:
        language = st.selectbox(
            "Code Language",
            ["Python", "JavaScript", "Java", "C++", "SQL"],
            key="exp_language"
        )

        selected_model = st.selectbox(
            "AI Model",
            list(MODELS.keys()),
            key='exp_model_key'
        )

        code_input = st.text_area(
            "Paste your code here:",
            height=300,
            placeholder=f"Paste your {language} code here...",
            key="code_input"
        )

        if st.button("Analyze Code", type="primary", use_container_width=True):
            if not code_input.strip():
                st.warning("Please paste some code to analyze.")
            else:
                with st.spinner("Analyzing code..."):
                    start_time = time.time()
                    try:
                        explanation, tokens_used = generate_explanation(code_input, selected_model, language)
                        end_time = time.time()
                        latency = end_time - start_time
                    except Exception as e:
                        explanation = f"Error: {str(e)}"
                        tokens_used = 0
                        latency = 0

                st.session_state.last_explanation = explanation
                st.session_state.explanation_tokens = tokens_used
                st.session_state.explanation_latency = latency

                # Create history entry for explainer
                user_id = get_user_id(st.session_state.username)
                history_inputs = {
                    "code": code_input,
                    "language": language,
                    "model": selected_model
                }
                history_metadata = {
                    "model": selected_model,
                    "latency_ms": round(latency * 1000, 2),
                    "tokens_used": tokens_used
                }

                create_history_entry(
                    user_id=user_id,
                    entry_type='explainer',
                    inputs=history_inputs,
                    outputs_text=explanation,
                    metadata=history_metadata
                )

                # Log activity
                log_activity(st.session_state.user_id, "code_explanation", "explainer",
                           f"Explained {language} code with {selected_model}")

    with col2:
        if hasattr(st.session_state, 'last_explanation'):
            st.subheader("Code Explanation")
            st.markdown(st.session_state.last_explanation)

            with st.expander("Analysis Details"):
                st.write(f"**Model:** {selected_model}")
                st.write(f"**Language:** {language}")
                st.write(f"**Response Time:** {st.session_state.explanation_latency:.2f}s")
                st.write(f"**Tokens Used:** {st.session_state.explanation_tokens}")

            # AST Visualization for Python
            if language == "Python":
                st.subheader("AST Visualization")
                visualizer = ASTVisualizer()
                graph = visualizer.get_graph(code_input)
                if graph:
                    st.graphviz_chart(graph)
                else:
                    st.info("Could not generate AST for this code.")

        else:
            st.info("Paste your code and click 'Analyze Code' to get a detailed explanation.")

    # Feedback system with star ratings
    if hasattr(st.session_state, 'last_explanation'):
        st.markdown("---")
        st.subheader("🌟 Rate This Explanation")

        # Use radio buttons for star rating (compatible with forms)
        rating = st.radio(
            "How helpful was this explanation?",
            options=[1, 2, 3, 4, 5],
            format_func=lambda x: f"{'⭐' * x} ({x}/5)",
            horizontal=True,
            key="exp_feedback_rating"
        )

        comment = st.text_area("Additional comments (optional):", placeholder="Any specific feedback or suggestions...", key="exp_feedback_comment")

        if st.button("Submit Feedback", use_container_width=True, key="exp_feedback_submit"):
            if rating > 0:
                # Create a conversation for the explanation
                user_id = get_user_id(st.session_state.username)
                conv_id = create_conversation(user_id, f"Code Explanation - {language}")
                message_id = add_message(
                    conv_id,
                    "assistant",
                    st.session_state.last_explanation,
                    selected_model,
                    language,
                    st.session_state.explanation_latency,
                    st.session_state.explanation_tokens
                )

                update_message_feedback(message_id, rating, comment)

                # Show thank you message with the rating
                rating_texts = {
                    1: "Poor ⭐",
                    2: "Fair ⭐⭐",
                    3: "Good ⭐⭐⭐",
                    4: "Very Good ⭐⭐⭐⭐",
                    5: "Excellent ⭐⭐⭐⭐⭐"
                }
                st.success(f"Thank you for your {rating_texts[rating]} feedback!")

                del st.session_state.last_explanation
                st.rerun()
            else:
                st.warning("Please select a star rating before submitting feedback.")

def show_chat_page():
    apply_theme()
    st.title("AI Chat Assistant")

    # Sidebar for chat settings
    with st.sidebar:
        st.subheader("Chat Settings")
        chat_model = st.selectbox(
            "Chat Model",
            list(CHAT_MODELS.keys()),
            key='chat_model_key'
        )

        if st.button("Clear History", use_container_width=True):
            st.session_state.chat_history = []
            # Also clear conversation if exists
            if 'chat_conversation_id' in st.session_state:
                delete_conversation(st.session_state.chat_conversation_id)
                del st.session_state.chat_conversation_id
            st.rerun()

    # Initialize chat history
    if 'chat_history' not in st.session_state:
        st.session_state.chat_history = []

    # Create conversation in database if user is logged in and doesn't have one
    if st.session_state.logged_in and 'chat_conversation_id' not in st.session_state:
        user_id = get_user_id(st.session_state.username)
        if user_id:
            conv_id = create_conversation(user_id, "AI Chat Conversation")
            st.session_state.chat_conversation_id = conv_id
            st.success("New chat conversation created!")

    # Display chat messages
    chat_container = st.container(height=500, border=True)
    with chat_container:
        for message in st.session_state.chat_history:
            with st.chat_message(message["role"]):
                st.markdown(message["content"])

    # Chat input
    user_input = st.chat_input("Ask me anything about programming...")

    if user_input:
        # Add user message to chat history
        st.session_state.chat_history.append({"role": "user", "content": user_input})

        # Save user message to database if logged in and conversation exists
        if st.session_state.logged_in and 'chat_conversation_id' in st.session_state and st.session_state.chat_conversation_id is not None:
            try:
                add_message(st.session_state.chat_conversation_id, "user", user_input, chat_model, "Chat")
            except Exception as e:
                st.error(f"Error saving message: {e}")
                # Create a new conversation if the current one is invalid
                user_id = get_user_id(st.session_state.username)
                if user_id:
                    conv_id = create_conversation(user_id, "AI Chat Conversation")
                    st.session_state.chat_conversation_id = conv_id
                    add_message(st.session_state.chat_conversation_id, "user", user_input, chat_model, "Chat")

        # Generate AI response
        with st.spinner("Thinking..."):
            start_time = time.time()
            try:
                response, tokens_used = chat_with_ai(user_input, chat_model)
                end_time = time.time()
                latency = end_time - start_time
            except Exception as e:
                response = f"I apologize, but I encountered an error: {str(e)}"
                tokens_used = 0
                latency = 0

        # Add AI response to chat history
        st.session_state.chat_history.append({"role": "assistant", "content": response})

        # Save AI response to database if logged in and conversation exists
        if st.session_state.logged_in and 'chat_conversation_id' in st.session_state and st.session_state.chat_conversation_id is not None:
            try:
                add_message(
                    st.session_state.chat_conversation_id,
                    "assistant",
                    response,
                    chat_model,
                    "Chat",
                    latency,
                    tokens_used
                )
            except Exception as e:
                st.error(f"Error saving AI response: {e}")
                # Create a new conversation if the current one is invalid
                user_id = get_user_id(st.session_state.username)
                if user_id:
                    conv_id = create_conversation(user_id, "AI Chat Conversation")
                    st.session_state.chat_conversation_id = conv_id
                    add_message(
                        st.session_state.chat_conversation_id,
                        "assistant",
                        response,
                        chat_model,
                        "Chat",
                        latency,
                        tokens_used
                    )

        # Log activity
        if st.session_state.logged_in:
            log_activity(st.session_state.user_id, "chat", "chat",
                        f"Chat with {chat_model}, tokens: {tokens_used}")

        st.rerun()

    # Enhanced Share button - only show if user is logged in and has messages and valid conversation
    if (st.session_state.logged_in and
        st.session_state.chat_history and
        'chat_conversation_id' in st.session_state and
        st.session_state.chat_conversation_id is not None):

        st.markdown("---")
        st.subheader("Share Chat")

        col1, col2 = st.columns([3, 1])

        with col1:
            if st.button("Generate Share Link", use_container_width=True, type="primary"):
                try:
                    share_id = share_conversation(st.session_state.chat_conversation_id)

                    # Generate the full shareable URL
                    base_url = "https://fossilisable-errol-arboresque.ngrok-free.dev"
                    share_url = f"{base_url}/?share_id={share_id}"

                    st.session_state.generated_share_url = share_url
                    st.session_state.generated_share_id = share_id
                    st.success("Share link generated successfully!")

                    # Log the sharing activity
                    log_activity(st.session_state.user_id, "share_chat", "chat",
                               f"Shared chat with ID: {share_id}")
                except Exception as e:
                    st.error(f"Error generating share link: {e}")

        with col2:
            if st.button("Unshare Chat", use_container_width=True):
                if 'chat_conversation_id' in st.session_state and st.session_state.chat_conversation_id is not None:
                    try:
                        unshare_conversation(st.session_state.chat_conversation_id)
                        if 'generated_share_url' in st.session_state:
                            del st.session_state.generated_share_url
                        if 'generated_share_id' in st.session_state:
                            del st.session_state.generated_share_id
                        st.success("Chat unshared successfully!")
                        st.rerun()
                    except Exception as e:
                        st.error(f"Error unsharing chat: {e}")

        # Display the generated share URL
        if hasattr(st.session_state, 'generated_share_url'):
            st.markdown("### Shareable Link")

            # Display the URL in a nice box
            st.code(st.session_state.generated_share_url, language=None)

            # Copy to clipboard functionality
            col_copy1, col_copy2, col_copy3 = st.columns([1, 1, 1])

            with col_copy1:
                if st.button("📋 Copy Link", use_container_width=True):
                    # For Streamlit, we can use the clipboard via JavaScript
                    js_code = f"""
                    <script>
                    function copyToClipboard() {{
                        navigator.clipboard.writeText("{st.session_state.generated_share_url}");
                    }}
                    copyToClipboard();
                    </script>
                    """
                    st.components.v1.html(js_code, height=0)
                    st.success("Link copied to clipboard!")

            with col_copy2:
                # Create a direct link that users can click
                st.markdown(
                    f'<a href="{st.session_state.generated_share_url}" target="_blank" style="text-decoration: none;">'
                    '<button style="width: 100%; padding: 0.5rem; border: none; border-radius: 0.375rem; background-color: #6366f1; color: white; cursor: pointer;">🔗 Open Link</button>'
                    '</a>',
                    unsafe_allow_html=True
                )

            with col_copy3:
                # QR code for mobile sharing (optional enhancement)
                if st.button("📱 QR Code", use_container_width=True):
                    try:
                        import qrcode
                        from io import BytesIO

                        # Generate QR code
                        qr = qrcode.QRCode(version=1, box_size=10, border=5)
                        qr.add_data(st.session_state.generated_share_url)
                        qr.make(fit=True)

                        img = qr.make_image(fill_color="black", back_color="white")
                        buffered = BytesIO()
                        img.save(buffered, format="PNG")

                        st.image(buffered, caption="Scan to open chat", use_column_width=True)
                    except ImportError:
                        st.info("Install 'qrcode' package for QR code generation: `pip install qrcode[pil]`")

            st.info("**💡 Pro Tip:** Share this link with others to let them view this chat conversation. They don't need to log in to see it!")

def show_shared_chat_page(share_id):
    apply_theme()

    # Header with nice styling
    st.markdown("""
    <div style='text-align: center; padding: 2rem 0;'>
        <h1 style='color: #6366f1; margin-bottom: 0.5rem;'>🔗 Shared Chat</h1>
        <p style='color: #6b7280; font-size: 1.1rem;'>View-only conversation shared from CodeGenie</p>
    </div>
    """, unsafe_allow_html=True)

    # Get shared conversation
    conv_data = get_shared_conversation(share_id)

    if not conv_data:
        st.error("""
        ## ❌ Shared Conversation Not Found

        This could be because:
        - The link has expired
        - The conversation was deleted
        - The sharing was turned off by the owner

        Please contact the person who shared this link with you.
        """)

        # Option to go to main app
        if st.button("🚀 Go to CodeGenie Home", use_container_width=True):
            # Clear the share_id from URL
            try:
                st.query_params.clear()
            except:
                st.experimental_set_query_params()
            st.rerun()
        return

    conversation_id, title, user_id = conv_data

    # Get messages from the conversation
    messages = get_conversation_messages(conversation_id)

    if not messages:
        st.info("""
        ## 💬 No Messages Yet

        This shared conversation doesn't have any messages yet.
        Check back later or contact the owner.
        """)
        return

    # Display conversation info
    col1, col2, col3 = st.columns([2, 1, 1])

    with col1:
        st.subheader(f"💬 {title}")

    with col2:
        st.metric("Total Messages", len(messages))

    with col3:
        # Count user vs assistant messages
        user_msgs = sum(1 for msg in messages if msg[0] == "user")
        assistant_msgs = sum(1 for msg in messages if msg[0] == "assistant")
        st.metric("Exchange", f"{user_msgs} : {assistant_msgs}")

    st.markdown("---")

    # Display messages in a beautiful chat interface
    chat_container = st.container()
    with chat_container:
        for i, (role, content, model_name, msg_language, timestamp, response_time, feedback_score, feedback_comment, tokens_used) in enumerate(messages):
            # Use columns for better layout
            col1, col2 = st.columns([1, 20])

            with col1:
                if role == "user":
                    st.markdown("👤")
                else:
                    st.markdown("🤖")

            with col2:
                # Message bubble
                if role == "user":
                    st.markdown(
                        f"""
                        <div style='
                            background: linear-gradient(135deg, #6366f1, #8b5cf6);
                            color: white;
                            padding: 1rem;
                            border-radius: 1rem 1rem 0.25rem 1rem;
                            margin: 0.5rem 0;
                            box-shadow: 0 2px 8px rgba(99, 102, 241, 0.3);
                        '>
                            {content}
                        </div>
                        """,
                        unsafe_allow_html=True
                    )
                else:
                    st.markdown(
                        f"""
                        <div style='
                            background-color: {'#1f2937' if st.session_state.theme == 'dark' else '#f3f4f6'};
                            color: {'#f9fafb' if st.session_state.theme == 'dark' else '#111827'};
                            padding: 1rem;
                            border-radius: 1rem 1rem 1rem 0.25rem;
                            margin: 0.5rem 0;
                            border: 1px solid {'#374151' if st.session_state.theme == 'dark' else '#d1d5db'};
                            box-shadow: 0 1px 3px rgba(0, 0, 0, 0.1);
                        '>
                            {content}
                        </div>
                        """,
                        unsafe_allow_html=True
                    )

                # Details expander
                with st.expander("📊 Details", expanded=False):
                    col_details1, col_details2 = st.columns(2)

                    with col_details1:
                        if timestamp:
                            st.caption(f"**Time:** {timestamp}")
                        if model_name:
                            st.caption(f"**Model:** {model_name}")

                    with col_details2:
                        if response_time:
                            st.caption(f"**Response Time:** {response_time:.2f}s")
                        if tokens_used:
                            st.caption(f"**Tokens Used:** {tokens_used}")
                        if feedback_score:
                            stars = "⭐" * feedback_score + "☆" * (5 - feedback_score)
                            st.caption(f"**Feedback:** {stars}")

    st.markdown("---")

    # Call to action section
    st.markdown("""
    <div style='
        background: linear-gradient(135deg, #f0f9ff, #e0f2fe);
        padding: 2rem;
        border-radius: 1rem;
        text-align: center;
        border: 1px solid #bae6fd;
        margin: 2rem 0;
    '>
        <h3 style='color: #0369a1; margin-bottom: 1rem;'>💡 Inspired by this chat?</h3>
        <p style='color: #0c4a6e; margin-bottom: 1.5rem;'>
            Create your own AI conversations with CodeGenie! Get code generation,
            explanations, and programming assistance.
        </p>
    </div>
    """, unsafe_allow_html=True)

    # Action buttons
    col_action1, col_action2, col_action3 = st.columns(3)

    with col_action1:
        if st.button("🚀 Try CodeGenie", use_container_width=True, type="primary"):
            # Clear share parameters and redirect to home
            try:
                st.query_params.clear()
            except:
                st.experimental_set_query_params()
            st.session_state.page = "Login"
            st.rerun()

    with col_action2:
        if st.button("📋 Copy This Link", use_container_width=True):
            # Get current URL for copying
            try:
                from streamlit.web.server.websocket_headers import _get_websocket_headers
                headers = _get_websocket_headers()
                if headers and 'Host' in headers:
                    current_url = f"http://{headers['Host']}/?share_id={share_id}"
                else:
                    current_url = f"http://localhost:8501/?share_id={share_id}"
            except:
                current_url = f"/?share_id={share_id}"

            # Copy to clipboard
            js_code = f"""
            <script>
            navigator.clipboard.writeText("{current_url}");
            </script>
            """
            st.components.v1.html(js_code, height=0)
            st.success("Link copied to clipboard!")

    with col_action3:
        # Direct link to open in new tab
        try:
            from streamlit.web.server.websocket_headers import _get_websocket_headers
            headers = _get_websocket_headers()
            if headers and 'Host' in headers:
                current_url = f"http://{headers['Host']}"
            else:
                current_url = "http://localhost:8501"
        except:
            current_url = ""

        if current_url:
            st.markdown(
                f'<a href="{current_url}" target="_blank" style="text-decoration: none;">'
                '<button style="width: 100%; padding: 0.75rem; border: 1px solid #d1d5db; border-radius: 0.5rem; background-color: white; color: #374151; cursor: pointer;">🏠 Visit Home</button>'
                '</a>',
                unsafe_allow_html=True
            )

def show_profile_page():
    apply_theme()
    st.title(f"Profile: {st.session_state.username}")

    col1, col2 = st.columns([1, 2])

    with col1:
        st.subheader("User Information")
        st.markdown(f"""
        <div style='padding: 20px; border-radius: 10px; background-color: {'#1E1E1E' if st.session_state.theme == 'dark' else '#F0F0F0'};'>
            <p><strong>Username:</strong> {st.session_state.username}</p>
            <p><strong>Email:</strong> {st.session_state.email}</p>
            <p><strong>Role:</strong> {st.session_state.role}</p>
            <p><strong>Theme:</strong> {st.session_state.theme.title()}</p>
        </div>
        """, unsafe_allow_html=True)

        st.subheader("Appearance")
        if st.button("Switch Theme", use_container_width=True):
            st.session_state.theme = 'light' if st.session_state.theme == 'dark' else 'dark'
            apply_theme()
            st.rerun()

    with col2:
        st.subheader("Security Settings")

        with st.form("change_password_form"):
            st.markdown("### Change Password")
            current_password = st.text_input("Current Password", type="password")
            new_password = st.text_input("New Password", type="password")
            confirm_password = st.text_input("Confirm New Password", type="password")

            if st.form_submit_button("Update Password", use_container_width=True):
                if not all([current_password, new_password, confirm_password]):
                    st.error("Please fill in all fields")
                elif new_password != confirm_password:
                    st.error("New passwords do not match")
                else:
                    user_data = verify_user(st.session_state.username, current_password)
                    if user_data:
                        update_user_password(st.session_state.username, new_password)
                        log_activity(st.session_state.user_id, "password_change", "profile")
                        st.success("Password updated successfully!")
                    else:
                        st.error("Current password is incorrect")

        st.markdown("---")
        st.subheader("Activity History")

        # Show recent conversations
        user_id = get_user_id(st.session_state.username)
        conversations = get_user_conversations(user_id)

        if conversations:
            for conv_id, title, created_at, updated_at, is_shared, share_id in conversations[:5]:
                col_a, col_b = st.columns([3, 1])
                with col_a:
                    st.write(f"**{title}**")
                    st.caption(f"Updated: {updated_at}")
                with col_b:
                    if st.button("Open", key=f"open_{conv_id}"):
                        st.session_state.current_conversation_id = conv_id
                        st.session_state.conversation_messages = get_conversation_messages(conv_id)
                        st.session_state.page = "Generator"
                        st.rerun()
        else:
            st.info("No conversations yet.")

# --- ADMIN PAGES ---

def show_admin_dashboard_page():
    apply_theme()

    if st.session_state.role != 'admin':
        st.error("Access denied. Admin privileges required.")
        return

    # Modern styling
    st.markdown("""
    <style>
    .stApp { background-color: #0f0f23; }
    h1 {
        color: #fafafa;
        font-weight: 700;
        letter-spacing: -0.025em;
    }
    .metric-card {
        padding: 15px;
        border-radius: 12px;
        border: 1px solid #3f3f46;
        background-color: #18181b;
        margin: 5px;
    }
    </style>
    """, unsafe_allow_html=True)

    # Boot animation
    if not st.session_state.admin_boot_animation_done:
        boot_placeholder = st.empty()
        messages = [
            "Initializing Admin Dashboard...",
            "Loading Analytics Engine...",
            "Syncing User Data...",
            "System Ready. Welcome, Admin."
        ]
        for msg in messages:
            boot_placeholder.info(msg)
            time.sleep(1)
        st.session_state.admin_boot_animation_done = True
        st.rerun()

    st.title("Admin Dashboard")
    st_autorefresh(interval=15000, key="admin_dash_refresh")

    # Key metrics
    total_users = get_total_users()
    total_queries = get_total_queries()
    user_growth = get_user_growth_data()
    language_usage = get_language_usage()
    module_usage = get_module_usage()
    top_users = get_top_users()
    feedback_stats = get_feedback_stats()

    # Metrics row
    col1, col2, col3, col4 = st.columns(4)

    with col1:
        st.metric("Total Users", total_users)
    with col2:
        st.metric("Total Queries", total_queries)
    with col3:
        avg_rating = feedback_stats[0] if feedback_stats and feedback_stats[0] else 0
        st.metric("Avg Rating", f"{avg_rating:.1f}/5")
    with col4:
        positive_feedback = feedback_stats[2] if feedback_stats else 0
        total_feedback = feedback_stats[1] if feedback_stats else 1
        rate = (positive_feedback / total_feedback * 100) if total_feedback > 0 else 0
        st.metric("Satisfaction", f"{rate:.1f}%")

    st.markdown("---")

    # Charts and analytics - FIXED INDENTATION
    tab1, tab2, tab3, tab4 = st.tabs(["Analytics", "Usage Stats", "Top Users", "System Logs"])

    with tab1:  # This should be indented under the function
        col1, col2 = st.columns(2)

        with col1:
            st.subheader("User Growth Over Time")

            user_growth = get_user_growth_data()

            if user_growth and len(user_growth) > 0:
                # Convert to DataFrame
                growth_df = pd.DataFrame(user_growth, columns=['Date', 'Daily_Users'])

                # Ensure Date is datetime type
                growth_df['Date'] = pd.to_datetime(growth_df['Date'])

                # Calculate cumulative sum
                growth_df['Cumulative_Users'] = growth_df['Daily_Users'].cumsum()

                # Create the chart
                fig = px.line(growth_df,
                             x='Date',
                             y='Cumulative_Users',
                             title='User Growth Over Time',
                             labels={'Cumulative_Users': 'Total Users', 'Date': 'Date'})

                # Add points to the line
                fig.update_traces(mode='lines+markers')

                # Customize layout
                fig.update_layout(
                    xaxis=dict(showgrid=True),
                    yaxis=dict(showgrid=True),
                    plot_bgcolor='rgba(0,0,0,0)',
                    paper_bgcolor='rgba(0,0,0,0)',
                )

                st.plotly_chart(fig, use_container_width=True)

                # Show data table
                with st.expander("📊 View Growth Data"):
                    st.dataframe(growth_df)

            else:
                # Fallback: Create manual growth data from existing users
                st.warning("No growth data found. Generating from existing users...")

                users = get_all_users()
                if users:
                    # Create manual growth data
                    growth_data = {}
                    for user_id, username, email, role, created_at, is_active in users:
                        if is_active:
                            date_part = created_at.split()[0]  # Get YYYY-MM-DD part
                            growth_data[date_part] = growth_data.get(date_part, 0) + 1

                    # Convert to list and sort by date
                    manual_growth = sorted([(date, count) for date, count in growth_data.items()])

                    if manual_growth:
                        growth_df = pd.DataFrame(manual_growth, columns=['Date', 'Daily_Users'])
                        growth_df['Date'] = pd.to_datetime(growth_df['Date'])
                        growth_df['Cumulative_Users'] = growth_df['Daily_Users'].cumsum()

                        fig = px.line(growth_df,
                                     x='Date',
                                     y='Cumulative_Users',
                                     title='User Growth (Generated from User Data)',
                                     labels={'Cumulative_Users': 'Total Users', 'Date': 'Date'})
                        fig.update_traces(mode='lines+markers')
                        st.plotly_chart(fig, use_container_width=True)

                        with st.expander("📊 View Generated Data"):
                            st.dataframe(growth_df)
                    else:
                        st.error("Could not generate growth data from users")
                else:
                    st.error("No users found to generate growth data")

        with col2:
            # Module usage chart
            if module_usage:
                module_df = pd.DataFrame(module_usage, columns=['Module', 'Count'])
                fig_module = px.pie(module_df, values='Count', names='Module',
                                  title="Module Usage Distribution")
                st.plotly_chart(fig_module, use_container_width=True)

    with tab2:  # This should also be indented
        col1, col2 = st.columns(2)

        with col1:
            # Language usage
            if language_usage:
                lang_df = pd.DataFrame(language_usage, columns=['Language', 'Count'])
                fig_lang = px.bar(lang_df, x='Language', y='Count',
                                title="Programming Language Usage")
                st.plotly_chart(fig_lang, use_container_width=True)

        with col2:
            # Feedback distribution
            if feedback_stats and feedback_stats[1] > 0:
                feedback_data = {'Type': ['Positive (4-5)', 'Neutral (3)', 'Negative (1-2)'],
                               'Count': [feedback_stats[2],
                                       feedback_stats[1] - feedback_stats[2] - (feedback_stats[1]//5),
                                       feedback_stats[1]//5]}
                feedback_df = pd.DataFrame(feedback_data)
                fig_feedback = px.pie(feedback_df, values='Count', names='Type',
                                    title="Feedback Distribution")
                st.plotly_chart(fig_feedback, use_container_width=True)

    with tab3:  # This should also be indented
        st.subheader("Top Active Users")
        if top_users:
            top_df = pd.DataFrame(top_users, columns=['Username', 'Message Count'])
            st.dataframe(top_df, use_container_width=True)
        else:
            st.info("No user activity data yet.")

    with tab4:  # This should also be indented
        st.subheader("Live System Logs")
        logs_placeholder = st.empty()

        # Display system logs
        logs_html = "<div style='height: 400px; overflow-y: scroll; background-color: #1E1E1E; padding: 10px; border-radius: 5px;'>"
        for log in st.session_state.system_logs[:20]:
            logs_html += f"<div style='padding: 5px; border-bottom: 1px solid #333; font-family: monospace;'>{log}</div>"
        logs_html += "</div>"

        logs_placeholder.markdown(logs_html, unsafe_allow_html=True)

        # Model status
        st.subheader("Model Cache Status")
        model_status = get_model_status()
        for model, status in model_status.items():
            st.write(f"{model}: {status}")

def show_admin_users_page():
    apply_theme()

    if st.session_state.role != 'admin':
        st.error("Access denied. Admin privileges required.")
        return

    st.title("User Management")

    tab1, tab2, tab3 = st.tabs(["User List", "Global Activity", "Search"])

    with tab1:
        st.subheader("Registered Users")
        users = get_all_users()

        if users:
            user_data = []
            for user_id, username, email, role, created_at, is_active in users:
                user_data.append({
                    "ID": user_id,
                    "Username": username,
                    "Email": email,
                    "Role": role,
                    "Created": created_at,
                    "Status": "Active" if is_active else "Inactive"
                })

            user_df = pd.DataFrame(user_data)
            st.dataframe(user_df, use_container_width=True)

            # User actions
            st.subheader("User Actions")
            col1, col2, col3 = st.columns(3)

            with col1:
                selected_user = st.selectbox("Select User",
                                           [f"{u[1]} (ID: {u[0]})" for u in users if u[5]])

            with col2:
                action = st.selectbox("Action", ["Promote to Admin", "Deactivate User"])

            with col3:
                if st.button("Execute Action", use_container_width=True):
                    user_id = int(selected_user.split("ID: ")[1].split(")")[0])

                    if action == "Promote to Admin":
                        success, message = promote_to_admin(user_id)
                        if success:
                            st.success(message)
                            st.rerun()
                        else:
                            st.error(message)
                    elif action == "Deactivate User":
                        delete_user(user_id)
                        st.success("User deactivated successfully")
                        st.rerun()

            # Admin count warning
            admin_count = sum(1 for u in users if u[3] == 'admin' and u[5])
            if admin_count >= 2:
                st.warning("Maximum admin limit (2) reached. Cannot promote more users to admin.")

        else:
            st.info("No users found.")

    with tab2:
        st.subheader("Global Activity Log")
        activities = get_global_activity()

        if activities:
            activity_data = []
            for username, activity_type, module, timestamp, details in activities:
                activity_data.append({
                    "User": username,
                    "Type": activity_type,
                    "Module": module,
                    "Timestamp": timestamp,
                    "Details": details[:100] + "..." if len(details) > 100 else details
                })

            activity_df = pd.DataFrame(activity_data)
            st.dataframe(activity_df, use_container_width=True)
        else:
            st.info("No activity logs found.")

    with tab3:
        st.subheader("Global Search")

        search_term = st.text_input("Search across users, code, and feedback")
        search_type = st.selectbox("Search in", ["All", "Users", "Code", "Feedback"])

        if st.button("Search", use_container_width=True):
            if search_term:
                # Basic search implementation
                st.info(f"Searching for '{search_term}' in {search_type}...")
                # Enhanced search would query the database with proper indexing
            else:
                st.warning("Please enter a search term")

# --- MAIN APP ROUTER ---

def main():
    st.set_page_config(
        page_title="CodeGenie - AI Code Assistant",
        page_icon=None,
        layout="wide",
        initial_sidebar_state="expanded"
    )

    # Initialize database and state
    is_first_user, admin_count = init_db()
    initialize_state()

    # Verify JWT token
    if st.session_state.token:
        payload = verify_jwt(st.session_state.token)
        if not payload:
            # Token invalid, clear session
            for key in list(st.session_state.keys()):
                del st.session_state[key]
            st.rerun()

    # Check for shared chat link (works for both logged in and logged out users)
    try:
        # Try new Streamlit API first
        query_params = st.query_params
        share_id = query_params.get("share_id", None)
    except:
        try:
            # Fallback to experimental API
            query_params = st.experimental_get_query_params()
            share_id = query_params.get("share_id", [None])[0] if query_params.get("share_id") else None
        except:
            share_id = None

    if share_id:
        show_shared_chat_page(share_id)
        return

    # Route pages
    if not st.session_state.logged_in:
        if st.session_state.page == "Forgot Password":
            show_forgot_password_page()
        else:
            show_login_page(is_first_user, admin_count)
    else:
        # Sidebar navigation
        with st.sidebar:
            st.title("CodeGenie")
            st.markdown(f"Welcome, **{st.session_state.username}**!")
            st.markdown(f"*Role: {st.session_state.role}*")
            st.markdown("---")

            # Navigation based on role
            if st.session_state.role == 'admin':
                nav_options = {
                    "Dashboard": "Dashboard",
                    "User Management": "Admin Users",
                    # "Code Generator": "Generator",
                    # "Code Explainer": "Explainer",
                    # "AI Chat": "Chat",
                    #"History": "History",
                    "Profile": "Profile"
                }
            else:
                nav_options = {
                    "Code Generator": "Generator",
                    "Code Explainer": "Explainer",
                    "AI Chat": "Chat",
                    "History": "History",
                    "Profile": "Profile"
                }

            for label, page in nav_options.items():
                if st.button(label, use_container_width=True,
                           type="primary" if st.session_state.page == page else "secondary"):
                    st.session_state.page = page
                    st.rerun()

            st.markdown("---")

            # System info
            if torch.cuda.is_available():
                st.success("GPU Accelerated")
            else:
                st.warning("CPU Mode")

            # Model Cache Status
            # st.markdown("### Model Status")
            # model_status = get_model_status()
            # for model, status in model_status.items():
            #     st.write(f"{model}: {status}")

            # if MODEL_CACHE['loaded_at']:
            #     st.caption(f"Cache loaded: {MODEL_CACHE['loaded_at'].strftime('%H:%M:%S')}")

            # st.markdown("---")

            # Logout button
            if st.button("Logout", use_container_width=True, type="secondary"):
                log_activity(st.session_state.user_id, "logout", "authentication")
                add_system_log(f"User '{st.session_state.username}' logged out.", "INFO")
                for key in list(st.session_state.keys()):
                    del st.session_state[key]
                st.rerun()

        # Main content routing
        if st.session_state.page == "Dashboard":
            show_admin_dashboard_page()
        elif st.session_state.page == "Admin Users":
            show_admin_users_page()
        elif st.session_state.page == "Generator":
            show_generator_page()
        elif st.session_state.page == "Explainer":
            show_explainer_page()
        elif st.session_state.page == "Chat":
            show_chat_page()
        elif st.session_state.page == "History":
            # History page with tabs
            st.title("My History")
            tab1, tab2 = st.tabs(["📁 Code Generator History", "📖 Code Explainer History"])
            with tab1:
                show_generator_history_page()
            with tab2:
                show_explainer_history_page()
        elif st.session_state.page == "Profile":
            show_profile_page()
        else:
            st.session_state.page = "Generator"
            st.rerun()

if __name__ == "__main__":
    main()

Overwriting app.py


In [3]:
# Cell 3: Launch the App (Final Version)
import os
import time
import subprocess
import socket
from google.colab import userdata
from pyngrok import ngrok

# --- Clean up any old ngrok/Streamlit processes ----
print("Cleaning up old ngrok and Streamlit processes...")
!pkill -f ngrok 2>/dev/null || echo "No old ngrok"
!kill -9 $(lsof -t -i:8501) 2>/dev/null || echo "No old Streamlit"

# ---- Authenticate ngrok ----
ngrok.set_auth_token(os.environ['NGROK_TOKEN'])

# ---- Helper to wait for Streamlit ----
def wait_for_port(host="localhost", port=8501, timeout=90):
    start = time.time()
    while time.time() - start < timeout:
        try:
            with socket.create_connection((host, port), timeout=2):
                return True
        except OSError:
            time.sleep(1)
    return False

# ---- Start Streamlit ----
print("Starting Streamlit on port 8501...")
# Start the process and pipe logs to files
process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"],
    stdout=open("streamlit.log", "w"),
    stderr=open("streamlit.err", "w")
)

# ---- Wait until Streamlit is ready ----
print("Waiting for Streamlit to start...")
if not wait_for_port():
    print("ERROR: Streamlit did not start. Check app logs.")
    # Print the error logs if it fails
    !echo "--- STDOUT ---"
    !cat streamlit.log
    !echo "--- STDERR ---"
    !cat streamlit.err
else:
    # ---- Open ngrok tunnel only after Streamlit is ready ----
    print("Opening ngrok tunnel...")
    tunnel = ngrok.connect(8501)
    print(f"\nSUCCESS: Your Streamlit app is live!\nURL: {tunnel.public_url}\n")

Cleaning up old ngrok and Streamlit processes...
^C
No old Streamlit
Starting Streamlit on port 8501...
Waiting for Streamlit to start...
Opening ngrok tunnel...

SUCCESS: Your Streamlit app is live!
URL: https://fossilisable-errol-arboresque.ngrok-free.dev

